## Widget to place order normally

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
from collections import defaultdict

def place_order(kite):
    orders = []
    placed_orders = []
    kite_orange = 'yellow'

    # Output Widgets
    basket_output = widgets.Output()
    msg_output = widgets.Output()
    status_output = widgets.Output()
    summary_output = widgets.Output()

    # Buy/Sell Buttons
    def create_side_buttons(default='BUY'):
        selected_side = default
        buy_btn = widgets.Button(description="BUY",
                                 layout=widgets.Layout(width='80px', margin='0 8px 0 0'))
        sell_btn = widgets.Button(description="SELL",
                                  layout=widgets.Layout(width='80px', margin='0 8px 0 0'))

        side_box = widgets.VBox([widgets.HBox([buy_btn, sell_btn])],
                                layout=widgets.Layout(margin='0 0 10px 0'))

        def update_styles(btn_clicked):
            nonlocal selected_side
            selected_side = btn_clicked.description
            buy_btn.style.button_color = '#2196f3' if selected_side == 'BUY' else '#ccc'
            sell_btn.style.button_color = 'red' if selected_side == 'SELL' else '#ccc'

        buy_btn.on_click(update_styles)
        sell_btn.on_click(update_styles)
        update_styles(widgets.Button(description=selected_side))
        return side_box, lambda: selected_side

    # Order Widget with Typeable Fields
    def create_order_widget():
        # Typeable Trading Symbol
        tradingsymbol = widgets.Text(
            value='',
            placeholder='Enter trading symbol (e.g., NIFTY25JAN26000CE)',
            description='Symbol:',
            layout=widgets.Layout(width='400px', margin='0 0 10px 0')
        )
        
        # Typeable Quantity
        quantity = widgets.IntText(
            value=15,
            description='Quantity:',
            layout=widgets.Layout(width='200px', margin='0 0 10px 0')
        )
        
        side_box, get_side = create_side_buttons('BUY')
        
        order_type = widgets.Dropdown(
            options=["MARKET", "LIMIT", "SL", "SL-M"],
            value="MARKET",
            description="Type:",
            layout=widgets.Layout(width='200px', margin='0 0 10px 0')
        )
        
        price_input = widgets.FloatText(
            value=0.0,
            description='Price:',
            layout=widgets.Layout(width='200px', margin='0 0 10px 0')
        )
        
        trigger_price_input = widgets.FloatText(
            value=0.0,
            description='Trigger:',
            layout=widgets.Layout(width='200px', margin='0 0 10px 0')
        )
        
        price_container = widgets.VBox([], layout=widgets.Layout(margin='0 0 10px 0'))
        
        def update_price_fields(change):
            order_type_val = change['new']
            if order_type_val == 'MARKET':
                price_container.children = []
            elif order_type_val == 'LIMIT':
                price_container.children = [price_input]
            elif order_type_val == 'SL':
                price_container.children = [trigger_price_input, price_input]
            elif order_type_val == 'SL-M':
                price_container.children = [trigger_price_input]
        
        order_type.observe(update_price_fields, names='value')
        update_price_fields({'new': order_type.value})
        
        exchange = widgets.Dropdown(
            options=["NSE", "NFO", "BSE", "CDS", "MCX"],
            value="NFO",
            description="Exchange:",
            layout=widgets.Layout(width='200px', margin='0 0 10px 0')
        )
        
        product = widgets.Dropdown(
            options=["MIS", "CNC", "NRML"],
            value="MIS",
            description="Product:",
            layout=widgets.Layout(width='200px', margin='0 0 10px 0')
        )
        
        variety = widgets.Dropdown(
            options=["regular", "amo"],
            value="regular",
            description="Variety:",
            layout=widgets.Layout(width='200px', margin='0 0 10px 0')
        )
        
        add_btn = widgets.Button(
            description="Add Order",
            button_style="primary",
            layout=widgets.Layout(margin='20px 0 20px 0')
        )

        return {
            "tradingsymbol": tradingsymbol,
            "quantity": quantity,
            "get_side": get_side,
            "side_box": side_box,
            "order_type": order_type,
            "price_input": price_input,
            "trigger_price_input": trigger_price_input,
            "price_container": price_container,
            "exchange": exchange,
            "product": product,
            "variety": variety,
            "add_btn": add_btn
        }

    order_widget = create_order_widget()

    # Add Order Logic
    def add_order_click(b):
        sym = order_widget["tradingsymbol"].value.strip().upper()
        
        if not sym:
            with msg_output:
                clear_output()
                print("⚠️ Please enter a trading symbol!")
            return
        
        qty = order_widget["quantity"].value
        if qty <= 0:
            with msg_output:
                clear_output()
                print("⚠️ Quantity must be greater than 0!")
            return
        
        txn_type = order_widget["get_side"]()
        order_type_val = order_widget["order_type"].value

        order = {
            "tradingsymbol": sym,
            "exchange": order_widget["exchange"].value,
            "transaction_type": txn_type,
            "quantity": qty,
            "order_type": order_type_val,
            "product": order_widget["product"].value,
            "variety": order_widget["variety"].value
        }
        
        if order_type_val == 'LIMIT':
            order['price'] = order_widget['price_input'].value
        elif order_type_val == 'SL':
            order['trigger_price'] = order_widget['trigger_price_input'].value
            order['price'] = order_widget['price_input'].value
        elif order_type_val == 'SL-M':
            order['trigger_price'] = order_widget['trigger_price_input'].value

        orders.append(order)

        with basket_output:
            clear_output()
            display(HTML("<h4 style='color: orange;'>🛒 Order Basket:</h4>"))
            for o in orders:
                side_color = "green" if o["transaction_type"] == "BUY" else "red"
                box_style = (
                    "display: inline-block; padding: 5px 10px; margin: 2px 4px; "
                    "border-radius: 5px; font-family: monospace; font-weight: bold; color: white;"
                )
                symbol_box = f"<span style='{box_style} background-color: #2196f3;'>{o['tradingsymbol']}</span>"
                qty_box_html = f"<span style='{box_style} background-color: #2196f3;'>{o['quantity']}</span>"
                side_box_html = f"<span style='{box_style} background-color: {side_color};'>{o['transaction_type']}</span>"
                type_box = f"<span style='{box_style} background-color: #2196f3;'>{o['order_type']}</span>"
                exchange_box = f"<span style='{box_style} background-color: #666;'>{o['exchange']}</span>"
                
                price_info = ""
                if 'price' in o:
                    price_info += f" @ ₹{o['price']:.2f}"
                if 'trigger_price' in o:
                    price_info += f" (Trigger: ₹{o['trigger_price']:.2f})"
                
                product_box = f"<span style='{box_style} background-color: #2196f3;'>{o['product']}</span>"
                variety_box = f"<span style='{box_style} background-color: #2196f3;'>{o['variety']}</span>"
                
                display(HTML(exchange_box + symbol_box + qty_box_html + side_box_html + type_box + 
                           f"<span style='color: #666; margin: 0 5px;'>{price_info}</span>" +
                           product_box + variety_box))

    order_widget["add_btn"].on_click(add_order_click)

    # Clear Orders
    clear_btn = widgets.Button(description="Clear Orders", button_style="danger")
    def clear_orders(b):
        orders.clear()
        placed_orders.clear()
        with basket_output:
            clear_output()
            print("🗑️ All orders cleared.")
        with msg_output:
            clear_output()
        with status_output:
            clear_output()
        with summary_output:
            clear_output()
    clear_btn.on_click(clear_orders)

    # Check Margin
    margin_btn = widgets.Button(description="Check Margin", button_style="info")
    def check_margin_required():
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to check margin for!")
            return None, None

        margin_orders = []
        for o in orders:
            margin_order = {
                "exchange": o["exchange"],
                "tradingsymbol": o["tradingsymbol"],
                "transaction_type": o["transaction_type"],
                "quantity": o["quantity"],
                "product": o["product"],
                "order_type": o["order_type"]
            }
            if 'price' in o:
                margin_order['price'] = o['price']
            if 'trigger_price' in o:
                margin_order['trigger_price'] = o['trigger_price']
            
            margin_orders.append(margin_order)

        try:
            margins = kite.margins()
            margin_info = kite.basket_order_margins(margin_orders, consider_positions=True, mode="compact")
            available_balance = margins['equity']['net']
            df = pd.json_normalize(margin_info['orders'], sep='_')
            total_required = df['total'].sum()

            with msg_output:
                clear_output()
                print(f"💰 Available Balance: ₹{available_balance:,.2f}")
                print(f"📊 Margin Required: ₹{total_required:,.2f}")
                if total_required > available_balance:
                    print("⚠️ Insufficient funds to place these orders!")
                else:
                    print("✅ Sufficient funds available.")

            return total_required, available_balance
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error checking margin: {e}")
            return None, None

    margin_btn.on_click(lambda b: check_margin_required())

    # Check Order Status
    def check_order_status():
        if not placed_orders:
            with status_output:
                clear_output()
                print("⚠️ No orders to check status for!")
            return

        with status_output:
            clear_output()
            display(HTML("<h4>📊 Order Status:</h4>"))
            
            for order_info in placed_orders:
                order_id = order_info['order_id']
                symbol = order_info['symbol']
                
                try:
                    order_history = kite.order_history(order_id=order_id)
                    
                    if order_history:
                        latest = order_history[-1]
                        status = latest.get('status', 'UNKNOWN')
                        status_msg = latest.get('status_message', '')
                        
                        if status == 'COMPLETE':
                            color = 'green'
                            icon = '✅'
                        elif status == 'REJECTED':
                            color = 'red'
                            icon = '❌'
                        elif status == 'CANCELLED':
                            color = 'orange'
                            icon = '🚫'
                        elif status in ['OPEN', 'TRIGGER PENDING']:
                            color = 'blue'
                            icon = '⏳'
                        else:
                            color = 'gray'
                            icon = '❓'
                        
                        display(HTML(
                            f"<pre style='color: {color}; font-weight: bold'>"
                            f"{icon} {symbol} (ID: {order_id}) → Status: {status}"
                            f"{' | Reason: ' + status_msg if status_msg else ''}</pre>"
                        ))
                        
                        order_info['status'] = status
                        order_info['status_message'] = status_msg
                        order_info['order_data'] = latest
                        
                except Exception as e:
                    display(HTML(
                        f"<pre style='color: red'>❌ Error fetching status for {symbol} (ID: {order_id}): {e}</pre>"
                    ))

    # Generate Execution Summary
    def generate_execution_summary():
        if not placed_orders:
            with summary_output:
                clear_output()
                print("⚠️ No orders to summarize!")
            return

        symbol_data = defaultdict(lambda: {
            'total_qty': 0,
            'total_amount': 0,
            'executed_qty': 0,
            'executed_amount': 0,
            'orders': []
        })

        for order_info in placed_orders:
            if 'order_data' not in order_info:
                continue
                
            order = order_info['order_data']
            symbol = order.get('tradingsymbol', order_info['symbol'])
            filled_qty = order.get('filled_quantity', 0)
            avg_price = order.get('average_price', 0)
            
            symbol_data[symbol]['total_qty'] += order.get('quantity', 0)
            symbol_data[symbol]['executed_qty'] += filled_qty
            
            if filled_qty > 0 and avg_price > 0:
                symbol_data[symbol]['executed_amount'] += filled_qty * avg_price
            
            symbol_data[symbol]['orders'].append(order_info)

        with summary_output:
            clear_output()
            display(HTML("<h4>📈 Execution Summary by Trading Symbol:</h4>"))
            
            if not symbol_data:
                print("No execution data available yet.")
                return
            
            for symbol, data in symbol_data.items():
                executed_qty = data['executed_qty']
                executed_amount = data['executed_amount']
                
                if executed_qty > 0:
                    avg_execution_price = executed_amount / executed_qty
                    
                    display(HTML(f"""
                        <div style='background-color: #f5f5f5; padding: 10px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #2196f3;'>
                            <h5 style='margin: 0 0 10px 0; color: #2196f3;'>{symbol}</h5>
                            <div style='display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px;'>
                                <div><strong>Executed Quantity:</strong> {executed_qty}</div>
                                <div><strong>Average Price:</strong> ₹{avg_execution_price:.2f}</div>
                                <div><strong>Total Amount:</strong> ₹{executed_amount:,.2f}</div>
                                <div><strong>Orders:</strong> {len(data['orders'])}</div>
                            </div>
                        </div>
                    """))
                else:
                    display(HTML(f"""
                        <div style='background-color: #fff3cd; padding: 10px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #ffc107;'>
                            <h5 style='margin: 0; color: #856404;'>{symbol}</h5>
                            <p style='margin: 5px 0 0 0; color: #856404;'>No execution yet (Pending/Rejected)</p>
                        </div>
                    """))

    # Execute Orders
    def _execute_orders():
        with msg_output:
            clear_output()
            print("🚀 Placing all orders...\n")

        placed_orders.clear()

        for order in orders:
            try:
                order_params = {
                    "variety": order["variety"],
                    "exchange": order["exchange"],
                    "tradingsymbol": order["tradingsymbol"],
                    "transaction_type": order["transaction_type"],
                    "quantity": order["quantity"],
                    "order_type": order["order_type"],
                    "product": order["product"]
                }
                
                if 'price' in order:
                    order_params['price'] = order['price']
                if 'trigger_price' in order:
                    order_params['trigger_price'] = order['trigger_price']
                
                order_id = kite.place_order(**order_params)
                
                placed_orders.append({
                    'order_id': order_id,
                    'symbol': order['tradingsymbol'],
                    'original_order': order
                })
                
                with msg_output:
                    display(HTML(f"<pre style='color: green; font-weight: bold'>✔ Order placed: {order['tradingsymbol']} → Order ID: {order_id}</pre>"))
                    
            except Exception as e:
                error_msg = str(e)
                placed_orders.append({
                    'order_id': None,
                    'symbol': order['tradingsymbol'],
                    'original_order': order,
                    'error': error_msg,
                    'status': 'FAILED'
                })
                
                with msg_output:
                    display(HTML(f"<pre style='color: red; font-weight: bold'>❌ Error placing {order['tradingsymbol']}: {error_msg}</pre>"))

        time.sleep(2)
        check_order_status()
        generate_execution_summary()

    # Place Orders
    place_btn = widgets.Button(description="Place All Orders", button_style="success")

    def place_orders(b):
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to place!")
            return

        total_required, available_balance = check_margin_required()
        if total_required is None:
            return

        if total_required > available_balance:
            yes_btn = widgets.Button(description="Yes", button_style="success")
            no_btn = widgets.Button(description="No", button_style="danger")
            confirm_box = widgets.HBox([yes_btn, no_btn])

            with msg_output:
                print(f"⚠️ Insufficient funds! Required: ₹{total_required:,.2f}, Available: ₹{available_balance:,.2f}")
                print("Do you still want to proceed with placing orders?")
                display(confirm_box)

            def proceed_yes(b):
                confirm_box.close()
                _execute_orders()

            def proceed_no(b):
                with msg_output:
                    clear_output()
                    print("❌ Order placement cancelled.")

            yes_btn.on_click(proceed_yes)
            no_btn.on_click(proceed_no)
        else:
            _execute_orders()

    place_btn.on_click(place_orders)

    # Refresh Status Button
    refresh_btn = widgets.Button(description="Refresh Status", button_style="info")
    def refresh_status(b):
        check_order_status()
        generate_execution_summary()
    refresh_btn.on_click(refresh_status)

    # Layout
    order_form = widgets.VBox([
        widgets.HTML("<h3>📝 Order Entry Form</h3>"),
        order_widget["tradingsymbol"],
        order_widget["quantity"],
        order_widget["side_box"],
        order_widget["exchange"],
        order_widget["order_type"],
        order_widget["price_container"],
        order_widget["product"],
        order_widget["variety"],
        order_widget["add_btn"]
    ], layout=widgets.Layout(padding='20px', border='2px solid #ddd', border_radius='10px'))

    # Display
    display(order_form)
    display(widgets.HBox([place_btn, clear_btn, margin_btn, refresh_btn], 
                         layout=widgets.Layout(margin='20px 0 0 0')))
    display(basket_output)
    display(msg_output)
    display(status_output)
    display(summary_output)

## This below code allows selecting direction of your choice
## then it picks up a hedge option and a future 

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML

def pick_direction(kite, instruments, lp, up):
    """
    Displays Bullish/Bearish strategy buttons, runs the selected strategy,
    and collects futures symbols, option symbols, and tokens.

    Returns:
    - fut_symbols: list
    - option_symbols: list
    - tokens: list
    """

    output = widgets.Output()

    # Global vars you want to collect
    global fut_symbols, option_symbols, tokens
    fut_symbols = []
    option_symbols = []
    tokens = []

    # ------------------------ BULLISH ------------------------
    def on_bullish_click(b):
        global fut_symbols, option_symbols, tokens

        # Your function returns ((symbol, token), (symbol, token))
        (fut_symbol, fut_token), (put_symbol, put_token) = bull_put_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )

        # Store cleaned values
        fut_symbols = [fut_symbol]
        option_symbols = [put_symbol]
        tokens = [fut_token, put_token]

        with output:
            output.clear_output()
            display(HTML(
                "<h3 style='color:green;'>🟢 BULLISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE UP 📈</h3>"
            ))
            print("▶ FUTURES symbol:", fut_symbols)
            print("▶ OPTION hedge symbol (PUT):", option_symbols)
            print("▶ TOKENS:", tokens)

    # ------------------------ BEARISH ------------------------
    def on_bearish_click(b):
        global fut_symbols, option_symbols, tokens

        # Your function returns ((symbol, token), (symbol, token))
        (fut_symbol, fut_token), (call_symbol, call_token) = bear_call_spread_with_future(
            kite, instruments, lower_premium=lp, upper_premium=up
        )

        # Store cleaned values
        fut_symbols = [fut_symbol]
        option_symbols = [call_symbol]
        tokens = [fut_token, call_token]

        with output:
            output.clear_output()
            display(HTML(
                "<h3 style='color:red;'>🔴 BEARISH STRATEGY SELECTED — PRICE EXPECTED TO MOVE DOWN 📉</h3>"
            ))
            print("▶ FUTURES symbol:", fut_symbols)
            print("▶ OPTION hedge symbol (CALL):", option_symbols)
            print("▶ TOKENS:", tokens)

    # ------------------------ BUTTONS ------------------------
    bullish_button = widgets.Button(description="Bullish Strategy", button_style='success')
    bearish_button = widgets.Button(description="Bearish Strategy", button_style='danger')

    bullish_button.on_click(on_bullish_click)
    bearish_button.on_click(on_bearish_click)

    display(widgets.VBox([widgets.HBox([bullish_button, bearish_button]), output]))

    # Return all 3 variables
    return fut_symbols, option_symbols, tokens


# This is the primary margin checking & order placing code

In [ ]:
def generate_qty_options(lot_size):
    return [lot_size * i for i in range(1, 7)]
qty_options = generate_qty_options(lot_size = 75)

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
from collections import defaultdict

def make_order_basket(fut_symbols, option_symbols, kite):
    orders = []
    placed_orders = []  # Track placed orders with IDs
    kite_orange = 'yellow'

    # -----------------------------
    # Output Widgets
    # -----------------------------
    basket_output = widgets.Output()
    msg_output = widgets.Output()
    status_output = widgets.Output()  # New: For order status tracking
    summary_output = widgets.Output()  # New: For execution summary

    # -----------------------------
    # Quantity Buttons
    # -----------------------------
    def create_qty_buttons():
        selected_qty = qty_options[0]
        buttons = []
        for q in qty_options:
            btn = widgets.Button(description=str(q),
                                 layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
            buttons.append(btn)

        qty_out = widgets.Output()
        qty_box = widgets.VBox([widgets.HBox(buttons + [qty_out])],
                               layout=widgets.Layout(margin='0 0 10px 0'))

        def update_styles(clicked_btn):
            nonlocal selected_qty
            selected_qty = int(clicked_btn.description)
            for b in buttons:
                b.style.button_color = kite_orange if b == clicked_btn else None
            with qty_out:
                clear_output()

        for b in buttons:
            b.on_click(update_styles)

        update_styles(buttons[0])
        return qty_box, lambda: selected_qty

    # -----------------------------
    # Buy/Sell Buttons
    # -----------------------------
    def create_side_buttons(default='BUY'):
        selected_side = default
        buy_btn = widgets.Button(description="BUY",
                                 layout=widgets.Layout(width='60px', margin='0 8px 0 0'))
        sell_btn = widgets.Button(description="SELL",
                                  layout=widgets.Layout(width='60px', margin='0 8px 0 0'))

        side_box = widgets.VBox([widgets.HBox([buy_btn, sell_btn])],
                                layout=widgets.Layout(margin='0 0 10px 0'))

        def update_styles(btn_clicked):
            nonlocal selected_side
            selected_side = btn_clicked.description
            buy_btn.style.button_color = '#2196f3' if selected_side == 'BUY' else '#ccc'
            sell_btn.style.button_color = 'red' if selected_side == 'SELL' else '#ccc'

        buy_btn.on_click(update_styles)
        sell_btn.on_click(update_styles)
        update_styles(widgets.Button(description=selected_side))
        return side_box, lambda: selected_side

    # -----------------------------
    # Order Widget Set with Dynamic Price Fields
    # -----------------------------
    def create_order_widget(symbols_list, default_buy=True):
        tradingsymbol = widgets.Dropdown(options=symbols_list, description="Symbol:",
                                         layout=widgets.Layout(margin='0 0 10px 0'))
        side_box, get_side = create_side_buttons('BUY' if default_buy else 'SELL')
        qty_box, get_qty = create_qty_buttons()
        order_type = widgets.Dropdown(options=["MARKET", "LIMIT","SL", "SL-M"], value="MARKET",
                                      description="Type:", layout=widgets.Layout(margin='0 0 10px 0'))
        
        price_input = widgets.FloatText(
            value=0.0,
            description='Price:',
            layout=widgets.Layout(margin='0 0 10px 0')
        )
        
        trigger_price_input = widgets.FloatText(
            value=0.0,
            description='Trigger:',
            layout=widgets.Layout(margin='0 0 10px 0')
        )
        
        price_container = widgets.VBox([], layout=widgets.Layout(margin='0 0 10px 0'))
        
        def update_price_fields(change):
            order_type_val = change['new']
            if order_type_val == 'MARKET':
                price_container.children = []
            elif order_type_val == 'LIMIT':
                price_container.children = [price_input]
            elif order_type_val == 'SL':
                price_container.children = [trigger_price_input, price_input]
            elif order_type_val == 'SL-M':
                price_container.children = [trigger_price_input]
        
        order_type.observe(update_price_fields, names='value')
        update_price_fields({'new': order_type.value})
        
        product = widgets.Dropdown(options=["MIS", "CNC", "NRML"], value="MIS",
                                   description="Product:", layout=widgets.Layout(margin='0 0 10px 0'))
        variety = widgets.Dropdown(options=["regular", "amo"], value="regular",
                                   description="Variety:", layout=widgets.Layout(margin='0 0 10px 0'))
        add_btn = widgets.Button(description="Add Order", button_style="primary",
                                 layout=widgets.Layout(margin='0 0 20px 0'))

        return {
            "tradingsymbol": tradingsymbol,
            "get_side": get_side,
            "side_box": side_box,
            "qty_box": qty_box,
            "get_qty": get_qty,
            "order_type": order_type,
            "price_input": price_input,
            "trigger_price_input": trigger_price_input,
            "price_container": price_container,
            "product": product,
            "variety": variety,
            "add_btn": add_btn
        }

    # -----------------------------
    # Determine default side for futures
    # -----------------------------
    default_fut_side = "BUY"
    if option_symbols and isinstance(option_symbols, list):
        first_opt = option_symbols[0]
        if first_opt.endswith("PE"):
            default_fut_side = "BUY"
        elif first_opt.endswith("CE"):
            default_fut_side = "SELL"

    fut_widget = create_order_widget(fut_symbols, default_buy=(default_fut_side=="BUY"))
    opt_widget = create_order_widget(option_symbols, default_buy=True)

    # -----------------------------
    # Add Order Logic
    # -----------------------------
    def add_order_widget(widget):
        sym = widget["tradingsymbol"].value
        txn_type = widget["get_side"]()
        qty = widget["get_qty"]()
        order_type_val = widget["order_type"].value

        if sym.endswith(("CE", "PE")):
            txn_type = "BUY"

        order = {
            "tradingsymbol": sym,
            "exchange": "NFO",
            "transaction_type": txn_type,
            "quantity": qty,
            "order_type": order_type_val,
            "product": widget["product"].value,
            "variety": widget["variety"].value
        }
        
        if order_type_val == 'LIMIT':
            order['price'] = widget['price_input'].value
        elif order_type_val == 'SL':
            order['trigger_price'] = widget['trigger_price_input'].value
            order['price'] = widget['price_input'].value
        elif order_type_val == 'SL-M':
            order['trigger_price'] = widget['trigger_price_input'].value

        orders.append(order)

        with basket_output:
            clear_output()
            display(HTML("<span style='color: orange; font-weight: bold;'>🛒 Order Basket:</span>"))
            for o in orders:
                side_color = "green" if o["transaction_type"] == "BUY" else "red"
                box_style = (
                    "display: inline-block; padding: 5px 10px; margin: 2px 4px; "
                    "border-radius: 5px; font-family: monospace; font-weight: bold; color: white;"
                )
                symbol_box = f"<span style='{box_style} background-color: #2196f3;'>{o['tradingsymbol']}</span>"
                qty_box_html = f"<span style='{box_style} background-color: #2196f3;'>{o['quantity']}</span>"
                side_box_html = f"<span style='{box_style} background-color: {side_color};'>{o['transaction_type']}</span>"
                type_box = f"<span style='{box_style} background-color: #2196f3;'>{o['order_type']}</span>"
                
                price_info = ""
                if 'price' in o:
                    price_info += f" @ ₹{o['price']:.2f}"
                if 'trigger_price' in o:
                    price_info += f" (Trigger: ₹{o['trigger_price']:.2f})"
                
                product_box = f"<span style='{box_style} background-color: #2196f3;'>{o['product']}</span>"
                variety_box = f"<span style='{box_style} background-color: #2196f3;'>{o['variety']}</span>"
                
                display(HTML(symbol_box + qty_box_html + side_box_html + type_box + 
                           f"<span style='color: #666; margin: 0 5px;'>{price_info}</span>" +
                           product_box + variety_box))

    fut_widget["add_btn"].on_click(lambda b: add_order_widget(fut_widget))
    opt_widget["add_btn"].on_click(lambda b: add_order_widget(opt_widget))

    # -----------------------------
    # Clear Orders
    # -----------------------------
    clear_btn = widgets.Button(description="Clear Orders", button_style="danger")
    def clear_orders(b):
        orders.clear()
        placed_orders.clear()
        with basket_output:
            clear_output()
            print("🗑️ All orders cleared.")
        with msg_output:
            clear_output()
        with status_output:
            clear_output()
        with summary_output:
            clear_output()
    clear_btn.on_click(clear_orders)

    # -----------------------------
    # Check Margin
    # -----------------------------
    margin_btn = widgets.Button(description="Check Margin", button_style="info")
    def check_margin_required():
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to check margin for!")
            return None, None

        margin_orders = []
        for o in orders:
            margin_order = {
                "exchange": "NFO",
                "tradingsymbol": o["tradingsymbol"],
                "transaction_type": o["transaction_type"],
                "quantity": o["quantity"],
                "product": o["product"],
                "order_type": o["order_type"]
            }
            if 'price' in o:
                margin_order['price'] = o['price']
            if 'trigger_price' in o:
                margin_order['trigger_price'] = o['trigger_price']
            
            margin_orders.append(margin_order)

        try:
            margins = kite.margins()
            margin_info = kite.basket_order_margins(margin_orders, consider_positions=True, mode="compact")
            available_balance = margins['equity']['net']
            df = pd.json_normalize(margin_info['orders'], sep='_')
            total_required = df['total'].sum()

            with msg_output:
                clear_output()
                print(f"💰 Available Balance: ₹{available_balance:,.2f}")
                print(f"📊 Margin Required: ₹{total_required:,.2f}")
                if total_required > available_balance:
                    print("⚠️ Insufficient funds to place these orders!")
                else:
                    print("✅ Sufficient funds available.")

            return total_required, available_balance
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error checking margin: {e}")
            return None, None

    margin_btn.on_click(lambda b: check_margin_required())

    # -----------------------------
    # NEW: Check Order Status
    # -----------------------------
    def check_order_status():
        if not placed_orders:
            with status_output:
                clear_output()
                print("⚠️ No orders to check status for!")
            return

        with status_output:
            clear_output()
            display(HTML("<h4>📊 Order Status:</h4>"))
            
            for order_info in placed_orders:
                order_id = order_info['order_id']
                symbol = order_info['symbol']
                
                try:
                    order_history = kite.order_history(order_id=order_id)
                    
                    if order_history:
                        latest = order_history[-1]
                        status = latest.get('status', 'UNKNOWN')
                        status_msg = latest.get('status_message', '')
                        
                        # Color code based on status
                        if status == 'COMPLETE':
                            color = 'green'
                            icon = '✅'
                        elif status == 'REJECTED':
                            color = 'red'
                            icon = '❌'
                        elif status == 'CANCELLED':
                            color = 'orange'
                            icon = '🚫'
                        elif status in ['OPEN', 'TRIGGER PENDING']:
                            color = 'blue'
                            icon = '⏳'
                        else:
                            color = 'gray'
                            icon = '❓'
                        
                        display(HTML(
                            f"<pre style='color: {color}; font-weight: bold'>"
                            f"{icon} {symbol} (ID: {order_id}) → Status: {status}"
                            f"{' | Reason: ' + status_msg if status_msg else ''}</pre>"
                        ))
                        
                        order_info['status'] = status
                        order_info['status_message'] = status_msg
                        order_info['order_data'] = latest
                        
                except Exception as e:
                    display(HTML(
                        f"<pre style='color: red'>❌ Error fetching status for {symbol} (ID: {order_id}): {e}</pre>"
                    ))

    # -----------------------------
    # NEW: Generate Execution Summary
    # -----------------------------
    def generate_execution_summary():
        if not placed_orders:
            with summary_output:
                clear_output()
                print("⚠️ No orders to summarize!")
            return

        # Aggregate by trading symbol
        symbol_data = defaultdict(lambda: {
            'total_qty': 0,
            'total_amount': 0,
            'executed_qty': 0,
            'executed_amount': 0,
            'orders': []
        })

        for order_info in placed_orders:
            if 'order_data' not in order_info:
                continue
                
            order = order_info['order_data']
            symbol = order.get('tradingsymbol', order_info['symbol'])
            filled_qty = order.get('filled_quantity', 0)
            avg_price = order.get('average_price', 0)
            
            symbol_data[symbol]['total_qty'] += order.get('quantity', 0)
            symbol_data[symbol]['executed_qty'] += filled_qty
            
            if filled_qty > 0 and avg_price > 0:
                symbol_data[symbol]['executed_amount'] += filled_qty * avg_price
            
            symbol_data[symbol]['orders'].append(order_info)

        with summary_output:
            clear_output()
            display(HTML("<h4>📈 Execution Summary by Trading Symbol:</h4>"))
            
            if not symbol_data:
                print("No execution data available yet.")
                return
            
            for symbol, data in symbol_data.items():
                executed_qty = data['executed_qty']
                executed_amount = data['executed_amount']
                
                if executed_qty > 0:
                    avg_execution_price = executed_amount / executed_qty
                    
                    display(HTML(f"""
                        <div style='background-color: #f5f5f5; padding: 10px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #2196f3;'>
                            <h5 style='margin: 0 0 10px 0; color: #2196f3;'>{symbol}</h5>
                            <div style='display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px;'>
                                <div><strong>Executed Quantity:</strong> {executed_qty}</div>
                                <div><strong>Average Price:</strong> ₹{avg_execution_price:.2f}</div>
                                <div><strong>Total Amount:</strong> ₹{executed_amount:,.2f}</div>
                                <div><strong>Orders:</strong> {len(data['orders'])}</div>
                            </div>
                        </div>
                    """))
                else:
                    display(HTML(f"""
                        <div style='background-color: #fff3cd; padding: 10px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #ffc107;'>
                            <h5 style='margin: 0; color: #856404;'>{symbol}</h5>
                            <p style='margin: 5px 0 0 0; color: #856404;'>No execution yet (Pending/Rejected)</p>
                        </div>
                    """))

    # -----------------------------
    # Internal function to execute orders
    # -----------------------------
    def _execute_orders():
        with msg_output:
            clear_output()
            print("🚀 Placing all orders...\n")

        placed_orders.clear()

        for order in orders:
            try:
                order_params = {
                    "variety": order["variety"],
                    "exchange": order["exchange"],
                    "tradingsymbol": order["tradingsymbol"],
                    "transaction_type": order["transaction_type"],
                    "quantity": order["quantity"],
                    "order_type": order["order_type"],
                    "product": order["product"]
                }
                
                if 'price' in order:
                    order_params['price'] = order['price']
                if 'trigger_price' in order:
                    order_params['trigger_price'] = order['trigger_price']
                
                order_id = kite.place_order(**order_params)
                
                placed_orders.append({
                    'order_id': order_id,
                    'symbol': order['tradingsymbol'],
                    'original_order': order
                })
                
                with msg_output:
                    display(HTML(f"<pre style='color: green; font-weight: bold'>✔ Order placed: {order['tradingsymbol']} → Order ID: {order_id}</pre>"))
                    
            except Exception as e:
                error_msg = str(e)
                placed_orders.append({
                    'order_id': None,
                    'symbol': order['tradingsymbol'],
                    'original_order': order,
                    'error': error_msg,
                    'status': 'FAILED'
                })
                
                with msg_output:
                    display(HTML(f"<pre style='color: red; font-weight: bold'>❌ Error placing {order['tradingsymbol']}: {error_msg}</pre>"))

        # Wait a moment for orders to process
        time.sleep(2)
        
        # Automatically check status after placement
        check_order_status()
        generate_execution_summary()

    # -----------------------------
    # Place Orders with insufficient funds confirmation
    # -----------------------------
    place_btn = widgets.Button(description="Place All Orders", button_style="success")

    def place_orders(b):
        if not orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders to place!")
            return

        total_required, available_balance = check_margin_required()
        if total_required is None:
            return

        if total_required > available_balance:
            yes_btn = widgets.Button(description="Yes", button_style="success")
            no_btn = widgets.Button(description="No", button_style="danger")
            confirm_box = widgets.HBox([yes_btn, no_btn])

            with msg_output:
                print(f"⚠️ Insufficient funds! Required: ₹{total_required:,.2f}, Available: ₹{available_balance:,.2f}")
                print("Do you still want to proceed with placing orders?")
                display(confirm_box)

            def proceed_yes(b):
                confirm_box.close()
                _execute_orders()

            def proceed_no(b):
                with msg_output:
                    clear_output()
                    print("❌ Order placement cancelled.")

            yes_btn.on_click(proceed_yes)
            no_btn.on_click(proceed_no)
        else:
            _execute_orders()

    place_btn.on_click(place_orders)

    # -----------------------------
    # NEW: Refresh Status Button
    # -----------------------------
    refresh_btn = widgets.Button(description="Refresh Status", button_style="info")
    def refresh_status(b):
        check_order_status()
        generate_execution_summary()
    refresh_btn.on_click(refresh_status)

    # -----------------------------
    # Layout FUTURES and OPTIONS
    # -----------------------------
    fut_box = widgets.VBox([
        widgets.HTML("<h4>FUTURES</h4>"),
        fut_widget["tradingsymbol"],
        fut_widget["side_box"],
        fut_widget["qty_box"],
        fut_widget["order_type"],
        fut_widget["price_container"],
        fut_widget["product"],
        fut_widget["variety"],
        fut_widget["add_btn"]
    ], layout=widgets.Layout(min_width='320px'))

    opt_box = widgets.VBox([
        widgets.HTML("<h4>OPTIONS</h4>"),
        opt_widget["tradingsymbol"],
        opt_widget["side_box"],
        opt_widget["qty_box"],
        opt_widget["order_type"],
        opt_widget["price_container"],
        opt_widget["product"],
        opt_widget["variety"],
        opt_widget["add_btn"]
    ], layout=widgets.Layout(min_width='320px'))

    container = widgets.HBox(
        [fut_box, opt_box],
        layout=widgets.Layout(
            justify_content='space-between',
            width='100%',
            padding='0 50px'
        )
    )

    # -----------------------------
    # Display everything
    # -----------------------------
    display(container)
    display(widgets.HBox([place_btn, clear_btn, margin_btn, refresh_btn], 
                         layout=widgets.Layout(margin='20px 0 0 0')))
    display(basket_output)
    display(msg_output)
    display(status_output)
    display(summary_output)

## Exit Positions with Full order panel

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

def exit_position_manager(kite):
    """
    Interactive position exit manager with multi-selection and customizable exit parameters.
    Supports both NSE and NFO exchanges with all order types.
    
    ✅ FIXED: Now uses kite.positions() API instead of order aggregation
    This correctly handles same tradingsymbol in different product types (CNC/MIS)
    
    Args:
        kite: KiteConnect instance
    """
    
    # Constants
    REQUIRED_COLUMNS = ['tradingsymbol', 'product', 'variety', 'exchange', 
                       'average_price', 'quantity', 'last_price', 'pnl']
    
    # State
    selected_positions = []  # Now a list for multi-select
    exit_orders = []
    placed_orders = []  # Track placed orders with status
    position_buttons = {}  # Store button references
    
    # Output widgets
    position_output = widgets.Output()
    exit_panel_output = widgets.Output()
    basket_output = widgets.Output()
    msg_output = widgets.Output()
    status_output = widgets.Output()
    summary_output = widgets.Output()
    
    # ========================================
    # STEP 1: Fetch and Display Positions (FIXED - Use Positions API)
    # ========================================
    def get_aggregated_positions():
        """
        Fetch positions from Kite Positions API
        ✅ This is the accurate way - directly from broker's position book
        """
        try:
            positions_data = kite.positions()
            
            # Combine net and day positions
            all_positions = []
            if positions_data.get('net'):
                all_positions.extend(positions_data['net'])
            if positions_data.get('day'):
                # Add day positions that aren't already in net
                net_keys = {(p['tradingsymbol'], p['product']) for p in positions_data.get('net', [])}
                for day_pos in positions_data['day']:
                    if (day_pos['tradingsymbol'], day_pos['product']) not in net_keys:
                        all_positions.append(day_pos)
            
            if not all_positions:
                return None
            
            positions_df = pd.DataFrame(all_positions)
            
            # Filter only open positions (quantity != 0)
            open_positions = positions_df[positions_df['quantity'] != 0].copy()
            
            if open_positions.empty:
                return None
            
            # Ensure all required columns exist
            for col in ['exchange', 'variety', 'last_price', 'pnl']:
                if col not in open_positions.columns:
                    if col == 'exchange':
                        open_positions[col] = 'NSE'  # Default
                    elif col == 'variety':
                        open_positions[col] = 'regular'
                    elif col == 'last_price':
                        open_positions[col] = open_positions['average_price']
                    elif col == 'pnl':
                        open_positions[col] = 0.0
            
            # Add exit parameters
            open_positions['OPEN_QTY'] = open_positions['quantity']
            open_positions['EXIT_QTY'] = open_positions['quantity'].abs()
            open_positions['exit_type'] = open_positions['quantity'].apply(
                lambda x: 'SELL' if x > 0 else 'BUY'
            )
            
            return open_positions
            
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error fetching positions: {e}")
            return None
    
    # ========================================
    # STEP 2: Display Position Buttons (Multi-select)
    # ========================================
    def display_position_buttons(positions_df):
        """Display positions as multi-selectable buttons"""
        nonlocal position_buttons
        position_buttons = {}
        
        with position_output:
            clear_output()
            display(HTML("<h3 style='color: #2196f3;'>📊 Active Positions - Select Positions to Exit</h3>"))
            
            buttons = []
            for idx, row in positions_df.iterrows():
                symbol = row['tradingsymbol']
                exchange = row.get('exchange', 'NFO')
                open_qty = row['OPEN_QTY']
                avg_price = row['average_price']
                product = row['product']
                pnl = row.get('pnl', 0)
                side = 'LONG' if open_qty > 0 else 'SHORT'
                side_color = 'green' if open_qty > 0 else 'red'
                pnl_color = 'green' if pnl >= 0 else 'red'
                
                # Create button label with product type and P&L
                label = f"{exchange}:{symbol} | {side} {abs(open_qty)} @ ₹{avg_price:.2f} [{product}] P&L: ₹{pnl:.2f}"
                
                btn = widgets.Button(
                    description=label,
                    layout=widgets.Layout(width='auto', margin='5px'),
                    style={'button_color': None}
                )
                
                # Store position data in button
                btn.position_data = row.to_dict()
                btn.position_id = f"{exchange}:{symbol}:{product}"  # Unique identifier with product
                btn.is_selected = False
                btn.on_click(lambda b: toggle_position_selection(b))
                
                buttons.append(btn)
                position_buttons[btn.position_id] = btn
            
            # Display buttons in rows
            rows = [buttons[i:i+2] for i in range(0, len(buttons), 2)]
            for row in rows:
                display(widgets.HBox(row))
            
            # Action buttons
            action_box = widgets.HBox([
                widgets.Button(description="🔄 Refresh Positions", button_style="info"),
                widgets.Button(description="✅ Confirm Selection", button_style="success"),
                widgets.Button(description="❌ Clear Selection", button_style="warning")
            ], layout=widgets.Layout(margin='15px 0 0 0'))
            
            refresh_btn, confirm_btn, clear_sel_btn = action_box.children
            
            refresh_btn.on_click(lambda b: initialize_ui())
            confirm_btn.on_click(lambda b: show_multi_exit_panel())
            clear_sel_btn.on_click(lambda b: clear_all_selections())
            
            display(action_box)
    
    def toggle_position_selection(btn):
        """Toggle position selection state"""
        nonlocal selected_positions
        
        if btn.is_selected:
            # Deselect
            btn.is_selected = False
            btn.style.button_color = None
            selected_positions = [p for p in selected_positions 
                                if f"{p.get('exchange', 'NFO')}:{p['tradingsymbol']}:{p['product']}" != btn.position_id]
        else:
            # Select
            btn.is_selected = True
            btn.style.button_color = 'lightgreen'
            selected_positions.append(btn.position_data)
        
        # Show count of selected positions
        with msg_output:
            clear_output()
            if selected_positions:
                symbols = [f"{p.get('exchange', 'NFO')}:{p['tradingsymbol']}[{p['product']}]" 
                          for p in selected_positions]
                print(f"✓ Selected {len(selected_positions)} position(s): {', '.join(symbols)}")
            else:
                print("No positions selected")
    
    def clear_all_selections():
        """Clear all position selections"""
        nonlocal selected_positions
        selected_positions = []
        
        for btn in position_buttons.values():
            btn.is_selected = False
            btn.style.button_color = None
        
        with msg_output:
            clear_output()
            print("All selections cleared")
        
        with exit_panel_output:
            clear_output()
    
    # ========================================
    # STEP 3: Multi-Position Exit Panel
    # ========================================
    def show_multi_exit_panel():
        """Display exit panel for multiple selected positions"""
        if not selected_positions:
            with msg_output:
                clear_output()
                print("⚠️ Please select at least one position first!")
            return
        
        with exit_panel_output:
            clear_output()
            
            # Show each position with individual controls
            for idx, position in enumerate(selected_positions):
                create_position_exit_widget(position, idx)
    
    def create_position_exit_widget(position, index):
        """Create exit widget for a single position"""
        symbol = position['tradingsymbol']
        exchange = position.get('exchange', 'NFO')
        open_qty = position['OPEN_QTY']
        exit_qty = position['EXIT_QTY']
        exit_type = position['exit_type']
        avg_price = position['average_price']
        product = position['product']
        
        side = 'LONG' if open_qty > 0 else 'SHORT'
        side_color = 'green' if open_qty > 0 else 'red'
        
        # Position header
        display(HTML(f"""
            <div style='background: #e3f2fd; padding: 12px; border-radius: 5px; margin: 15px 0 10px 0; border-left: 4px solid {side_color};'>
                <strong style='font-size: 16px;'>{exchange}:{symbol}</strong> | 
                <span style='color: {side_color}; font-weight: bold;'>{side} {abs(open_qty)}</span> @ ₹{avg_price:.2f} | 
                Product: <span style='background: #2196f3; color: white; padding: 2px 6px; border-radius: 3px;'>{product}</span> | 
                Exit: <span style='color: {"red" if exit_type == "SELL" else "green"}; font-weight: bold;'>{exit_type}</span>
            </div>
        """))
        
        # Create widgets for this position
        exchange_dropdown = widgets.Dropdown(
            options=['NSE', 'NFO', 'BSE', 'BFO', 'MCX', 'CDS'],
            value=exchange,
            description='Exchange:',
            layout=widgets.Layout(width='200px')
        )
        
        quantity_input = widgets.IntText(
            value=int(exit_qty),
            description='Quantity:',
            layout=widgets.Layout(width='200px')
        )
        
        order_type = widgets.Dropdown(
            options=['MARKET', 'LIMIT', 'SL', 'SL-M'],
            value='SL-M',
            description='Order Type:',
            layout=widgets.Layout(width='200px')
        )
        
        transaction_type = widgets.Dropdown(
            options=['BUY', 'SELL'],
            value=exit_type,
            description='Transaction:',
            layout=widgets.Layout(width='200px')
        )
        
        price_input = widgets.FloatText(
            value=round(avg_price, 2),
            description='Price:',
            layout=widgets.Layout(width='200px')
        )
        
        trigger_price_input = widgets.FloatText(
            value=round(avg_price + 10, 2) if exit_type == 'SELL' else round(avg_price - 10, 2),
            description='Trigger:',
            layout=widgets.Layout(width='200px')
        )
        
        stoploss_points = widgets.FloatText(
            value=10.0,
            description='SL Points:',
            layout=widgets.Layout(width='150px', margin='0 5px 0 0')
        )
        
        calc_sl_btn = widgets.Button(
            description="Revise",
            button_style="info",
            layout=widgets.Layout(width='100px', margin='0')
        )
        
        def calculate_stoploss(b):
            sl_points = stoploss_points.value
            current_transaction = transaction_type.value
            if current_transaction == 'SELL':
                # For SELL exits (closing LONG), trigger below avg price
                trigger_price_input.value = round(avg_price - sl_points, 2)
            else:
                # For BUY exits (closing SHORT), trigger above avg price
                trigger_price_input.value = round(avg_price + sl_points, 2)
            # For SL orders, limit price is usually same as trigger or slightly better
            price_input.value = trigger_price_input.value
        
        calc_sl_btn.on_click(calculate_stoploss)
        
        product_widget = widgets.Dropdown(
            options=['MIS', 'CNC', 'NRML'],
            value=product,  # Use the actual product from position
            description='Product:',
            layout=widgets.Layout(width='200px')
        )
        
        variety = widgets.Dropdown(
            options=['regular', 'amo', 'co', 'iceberg', 'auction'],
            value=position.get('variety', 'regular'),
            description='Variety:',
            layout=widgets.Layout(width='200px')
        )
        
        validity = widgets.Dropdown(
            options=['DAY', 'IOC', 'TTL'],
            value='DAY',
            description='Validity:',
            layout=widgets.Layout(width='200px')
        )
        
        # Disclosed quantity (for iceberg orders)
        disclosed_qty = widgets.IntText(
            value=0,
            description='Disclosed:',
            layout=widgets.Layout(width='200px')
        )
        
        # Tag field for custom reference
        tag = widgets.Text(
            value='',
            description='Tag:',
            placeholder='Optional reference',
            layout=widgets.Layout(width='200px')
        )
        
        # Price container (dynamic visibility)
        price_container = widgets.VBox([])
        
        def update_price_fields(change):
            order_type_val = change['new']
            if order_type_val == 'MARKET':
                price_container.children = []
            elif order_type_val == 'LIMIT':
                price_container.children = [price_input]
            elif order_type_val in ['SL', 'SL-M']:
                sl_calc_box = widgets.HBox(
                    [stoploss_points, calc_sl_btn],
                    layout=widgets.Layout(margin='0 0 5px 0')
                )
                if order_type_val == 'SL':
                    price_container.children = [
                        sl_calc_box,
                        trigger_price_input,
                        price_input
                    ]
                else:
                    price_container.children = [
                        sl_calc_box,
                        trigger_price_input
                    ]
        
        order_type.observe(update_price_fields, names='value')
        update_price_fields({'new': order_type.value})
        
        # Layout controls in columns
        col1 = widgets.VBox([
            exchange_dropdown,
            quantity_input,
            transaction_type,
            order_type
        ])
        
        col2 = widgets.VBox([
            product_widget,
            variety,
            validity,
            disclosed_qty
        ])
        
        col3 = widgets.VBox([
            price_container,
            tag
        ])
        
        control_box = widgets.HBox([col1, col2, col3], layout=widgets.Layout(margin='0 0 10px 0'))
        display(control_box)
        
        # Add to basket button for this position
        add_btn = widgets.Button(
            description=f"➕ Add {symbol}[{product}] to Basket",
            button_style="primary",
            layout=widgets.Layout(margin='0 0 15px 0')
        )
        
        def add_to_basket(b):
            order = build_order(position, exchange_dropdown, quantity_input, transaction_type,
                              order_type, price_input, trigger_price_input, product_widget, 
                              variety, validity, disclosed_qty, tag)
            exit_orders.append(order)
            display_basket()
            with msg_output:
                clear_output()
                print(f"✅ Added {exchange_dropdown.value}:{symbol}[{product_widget.value}] exit order to basket")
        
        add_btn.on_click(add_to_basket)
        display(add_btn)
        
        # Divider
        if index < len(selected_positions) - 1:
            display(HTML("<hr style='border: none; border-top: 1px solid #ddd; margin: 20px 0;'>"))
    
    def build_order(position, exchange_widget, qty_widget, transaction_widget,
                   order_type_widget, price_widget, trigger_widget, product_widget, 
                   variety_widget, validity_widget, disclosed_widget, tag_widget):
        """Build order dictionary from widgets"""
        order = {
            'tradingsymbol': position['tradingsymbol'],
            'exchange': exchange_widget.value,
            'transaction_type': transaction_widget.value,
            'quantity': int(qty_widget.value),
            'order_type': order_type_widget.value,
            'product': product_widget.value,
            'variety': variety_widget.value,
            'validity': validity_widget.value
        }
        
        # Add disclosed quantity if specified
        if disclosed_widget.value > 0:
            order['disclosed_quantity'] = int(disclosed_widget.value)
        
        # Add tag if specified
        if tag_widget.value.strip():
            order['tag'] = tag_widget.value.strip()
        
        # Add price fields based on order type
        if order_type_widget.value == 'LIMIT':
            order['price'] = float(price_widget.value)
        elif order_type_widget.value == 'SL':
            order['price'] = float(price_widget.value)
            order['trigger_price'] = float(trigger_widget.value)
        elif order_type_widget.value == 'SL-M':
            order['trigger_price'] = float(trigger_widget.value)
        
        return order
    
    # ========================================
    # Quick Add All to Basket
    # ========================================
    def add_all_to_basket_with_defaults():
        """Add all selected positions to basket with default SL settings"""
        if not selected_positions:
            with msg_output:
                clear_output()
                print("⚠️ No positions selected!")
            return
        
        for position in selected_positions:
            exit_type = position['exit_type']
            avg_price = position['average_price']
            exchange = position.get('exchange', 'NFO')
            
            # Default SL order
            order = {
                'tradingsymbol': position['tradingsymbol'],
                'exchange': exchange,
                'transaction_type': exit_type,
                'quantity': int(position['EXIT_QTY']),
                'order_type': 'SL',
                'product': position['product'],
                'variety': position.get('variety', 'regular'),
                'validity': 'DAY',
                'trigger_price': round(avg_price - 10, 2) if exit_type == 'SELL' else round(avg_price + 10, 2),
                'price': round(avg_price - 10, 2) if exit_type == 'SELL' else round(avg_price + 10, 2)
            }
            exit_orders.append(order)
        
        display_basket()
        with msg_output:
            clear_output()
            print(f"✅ Added {len(selected_positions)} positions to basket with default SL")
    
    # ========================================
    # STEP 4: Basket Management
    # ========================================
    def display_basket():
        """Display exit order basket"""
        with basket_output:
            clear_output()
            if not exit_orders:
                return
            
            display(HTML("<h3 style='color: #ff9800;'>🛒 Exit Order Basket</h3>"))
            
            for idx, order in enumerate(exit_orders):
                side_color = 'green' if order['transaction_type'] == 'BUY' else 'red'
                
                order_html = f"""
                    <div style='background: #f5f5f5; padding: 10px; margin: 5px 0; border-radius: 5px; border-left: 4px solid {side_color};'>
                        <strong>{order['exchange']}:{order['tradingsymbol']}</strong> | 
                        <span style='color: {side_color}; font-weight: bold;'>{order['transaction_type']} {order['quantity']}</span> | 
                        {order['order_type']} | {order['product']} | {order['validity']}
                        {f" @ ₹{order.get('price', 0):.2f}" if 'price' in order else ""}
                        {f" (Trigger: ₹{order.get('trigger_price', 0):.2f})" if 'trigger_price' in order else ""}
                        {f" [Tag: {order['tag']}]" if 'tag' in order else ""}
                    </div>
                """
                display(HTML(order_html))
            
            # Basket actions
            clear_basket_btn = widgets.Button(
                description="🗑️ Clear Basket",
                button_style="danger",
                layout=widgets.Layout(margin='10px 5px 0 0')
            )
            
            place_all_btn = widgets.Button(
                description="🚀 Place All Orders",
                button_style="success",
                layout=widgets.Layout(margin='10px 5px 0 0')
            )
            
            refresh_btn = widgets.Button(
                description="🔄 Refresh Status",
                button_style="info",
                layout=widgets.Layout(margin='10px 0 0 0')
            )
            
            def clear_basket(b):
                exit_orders.clear()
                placed_orders.clear()
                display_basket()
                with msg_output:
                    clear_output()
                    print("🗑️ Basket cleared")
                with status_output:
                    clear_output()
                with summary_output:
                    clear_output()
            
            def place_all(b):
                place_all_orders()
            
            def refresh_status(b):
                check_order_status()
                generate_execution_summary()
            
            clear_basket_btn.on_click(clear_basket)
            place_all_btn.on_click(place_all)
            refresh_btn.on_click(refresh_status)
            
            display(widgets.HBox([place_all_btn, clear_basket_btn, refresh_btn]))
    
    # ========================================
    # STEP 5: Order Status Management
    # ========================================
    def check_order_status():
        """Check status of all placed orders"""
        if not placed_orders:
            with status_output:
                clear_output()
                print("⚠️ No orders to check status for!")
            return

        with status_output:
            clear_output()
            display(HTML("<h4 style='color: #2196f3;'>📊 Order Status:</h4>"))
            
            for order_info in placed_orders:
                order_id = order_info['order_id']
                symbol = order_info['symbol']
                exchange = order_info.get('exchange', 'NFO')
                product = order_info.get('product', 'MIS')
                
                if order_id is None:
                    # Order failed to place
                    display(HTML(
                        f"<pre style='color: red; font-weight: bold'>❌ {exchange}:{symbol}[{product}] → FAILED TO PLACE</pre>"
                    ))
                    continue
                
                try:
                    order_history = kite.order_history(order_id=order_id)
                    
                    if order_history:
                        latest = order_history[-1]
                        status = latest.get('status', 'UNKNOWN')
                        status_msg = latest.get('status_message', '')
                        
                        if status == 'COMPLETE':
                            color = 'green'
                            icon = '✅'
                        elif status == 'REJECTED':
                            color = 'red'
                            icon = '❌'
                        elif status == 'CANCELLED':
                            color = 'orange'
                            icon = '🚫'
                        elif status in ['OPEN', 'TRIGGER PENDING']:
                            color = 'blue'
                            icon = '⏳'
                        else:
                            color = 'gray'
                            icon = '❓'
                        
                        display(HTML(
                            f"<pre style='color: {color}; font-weight: bold'>"
                            f"{icon} {exchange}:{symbol}[{product}] (ID: {order_id}) → Status: {status}"
                            f"{' | Reason: ' + status_msg if status_msg else ''}</pre>"
                        ))
                        
                        order_info['status'] = status
                        order_info['status_message'] = status_msg
                        order_info['order_data'] = latest
                        
                except Exception as e:
                    display(HTML(
                        f"<pre style='color: red'>❌ Error fetching status for {exchange}:{symbol}[{product}] (ID: {order_id}): {e}</pre>"
                    ))

    def generate_execution_summary():
        """Generate summary of executed orders by symbol"""
        if not placed_orders:
            with summary_output:
                clear_output()
                print("⚠️ No orders to summarize!")
            return

        from collections import defaultdict
        symbol_data = defaultdict(lambda: {
            'total_qty': 0,
            'total_amount': 0,
            'executed_qty': 0,
            'executed_amount': 0,
            'orders': []
        })

        for order_info in placed_orders:
            if 'order_data' not in order_info:
                continue
                
            order = order_info['order_data']
            symbol = order.get('tradingsymbol', order_info['symbol'])
            exchange = order.get('exchange', order_info.get('exchange', 'NFO'))
            product = order.get('product', order_info.get('product', 'MIS'))
            filled_qty = order.get('filled_quantity', 0)
            avg_price = order.get('average_price', 0)
            
            key = f"{exchange}:{symbol}[{product}]"
            
            symbol_data[key]['total_qty'] += order.get('quantity', 0)
            symbol_data[key]['executed_qty'] += filled_qty
            
            if filled_qty > 0 and avg_price > 0:
                symbol_data[key]['executed_amount'] += filled_qty * avg_price
            
            symbol_data[key]['orders'].append(order_info)

        with summary_output:
            clear_output()
            display(HTML("<h4 style='color: #2196f3;'>📈 Execution Summary by Trading Symbol:</h4>"))
            
            if not symbol_data:
                print("No execution data available yet.")
                return
            
            for symbol_key, data in symbol_data.items():
                executed_qty = data['executed_qty']
                executed_amount = data['executed_amount']
                
                if executed_qty > 0:
                    avg_execution_price = executed_amount / executed_qty
                    
                    display(HTML(f"""
                        <div style='background-color: #f5f5f5; padding: 10px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #2196f3;'>
                            <h5 style='margin: 0 0 10px 0; color: #2196f3;'>{symbol_key}</h5>
                            <div style='display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px;'>
                                <div><strong>Executed Quantity:</strong> {executed_qty}</div>
                                <div><strong>Average Price:</strong> ₹{avg_execution_price:.2f}</div>
                                <div><strong>Total Amount:</strong> ₹{executed_amount:,.2f}</div>
                                <div><strong>Orders:</strong> {len(data['orders'])}</div>
                            </div>
                        </div>
                    """))
                else:
                    display(HTML(f"""
                        <div style='background-color: #fff3cd; padding: 10px; margin: 10px 0; border-radius: 5px; border-left: 4px solid #ffc107;'>
                            <h5 style='margin: 0; color: #856404;'>{symbol_key}</h5>
                            <p style='margin: 5px 0 0 0; color: #856404;'>No execution yet (Pending/Rejected)</p>
                        </div>
                    """))

    # ========================================
    # STEP 6: Order Placement
    # ========================================
    def place_single_order(order):
        """Place a single order"""
        try:
            order_params = {k: v for k, v in order.items()}
            
            order_id = kite.place_order(**order_params)
            
            placed_orders.append({
                'order_id': order_id,
                'symbol': order['tradingsymbol'],
                'exchange': order['exchange'],
                'product': order['product'],
                'original_order': order
            })
            
            with msg_output:
                display(HTML(f"""
                    <div style='color: green; font-weight: bold; padding: 8px; background: #e8f5e9; border-radius: 5px; margin: 3px 0;'>
                        ✅ {order['exchange']}:{order['tradingsymbol']}[{order['product']}]: Order ID {order_id}
                    </div>
                """))
        except Exception as e:
            error_msg = str(e)
            placed_orders.append({
                'order_id': None,
                'symbol': order['tradingsymbol'],
                'exchange': order['exchange'],
                'product': order['product'],
                'original_order': order,
                'error': error_msg,
                'status': 'FAILED'
            })
            
            with msg_output:
                display(HTML(f"""
                    <div style='color: red; font-weight: bold; padding: 8px; background: #ffebee; border-radius: 5px; margin: 3px 0;'>
                        ❌ {order['exchange']}:{order['tradingsymbol']}[{order['product']}]: {error_msg}
                    </div>
                """))
    
    def place_all_orders(self):
        """Place all orders in basket"""
        if not exit_orders:
            with msg_output:
                clear_output()
                print("⚠️ No orders in basket!")
            return
        
        with msg_output:
            clear_output()
            print(f"🚀 Placing {len(exit_orders)} orders...\n")
        
        placed_orders.clear()
        
        for order in exit_orders:
            place_single_order(order)
        
        exit_orders.clear()
        display_basket()
        
        # Wait a moment for orders to be processed
        import time
        time.sleep(2)
        
        # Check status and generate summary
        check_order_status()
        generate_execution_summary()
    
    # ========================================
    # Initialize UI
    # ========================================
    def initialize_ui():
        """Initialize the complete UI"""
        nonlocal selected_positions
        selected_positions = []
        
        positions_df = get_aggregated_positions()
        
        with position_output:
            clear_output()
            if positions_df is None:
                print("⚠️ No active positions found")
                return
        
        with exit_panel_output:
            clear_output()
        
        with msg_output:
            clear_output()
        
        with status_output:
            clear_output()
        
        with summary_output:
            clear_output()
        
        display_position_buttons(positions_df)
    
    # Start the UI
    display(position_output)
    display(exit_panel_output)
    display(basket_output)
    display(msg_output)
    display(status_output)
    display(summary_output)
    
    initialize_ui()


# Usage example:
# exit_position_manager(kite)

# exiting positions with trailing stoploss and immediate exit but this one calle PAIs instead of kite ticker

# exiting positions with trailing stoploss and immediate exit but this one calle PAIs instead of kite ticker

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
import threading

def exit_manager_with_trail_sl_no_ticker(kite):
    """
    Interactive position manager with Trailing Stop Loss and Immediate Exit options.
    
    Features:
    - Select any active position
    - Trail Stop Loss: Automated trailing mechanism (BUY default, inverse for SELL)
    - Exit Immediately: Market order exit
    
    Args:
        kite: KiteConnect instance
    """
    
    # Constants
    EXECUTED_STATUSES = ['COMPLETE', '']
    REQUIRED_COLUMNS = ['status', 'tradingsymbol', 'order_type', 'variety', 'transaction_type', 
                       'product', 'quantity', 'price', 'average_price', 'exchange']
    
    # State
    selected_position = None
    trailing_threads = {}  # Track active trailing threads
    stop_trailing = {}  # Control flags for stopping threads
    
    # Output widgets
    position_output = widgets.Output()
    action_panel_output = widgets.Output()
    trail_config_output = widgets.Output()
    trail_status_output = widgets.Output()
    msg_output = widgets.Output()
    
    # ========================================
    # STEP 1: Fetch Positions
    # ========================================
    def get_aggregated_positions():
        """Fetch orders and calculate aggregated positions"""
        try:
            orders_df = pd.DataFrame(kite.orders())
            
            if orders_df.empty:
                return None
            
            executed_orders = orders_df[orders_df['status'].isin(EXECUTED_STATUSES)][REQUIRED_COLUMNS].copy()
            
            if executed_orders.empty:
                return None
            
            grouped = executed_orders.groupby(
                ['tradingsymbol', 'order_type', 'transaction_type', 'product', 'variety', 'exchange'],
                as_index=False
            ).agg({
                'price': 'mean',
                'average_price': 'mean',
                'quantity': 'sum'
            })
            
            agg_df = grouped.groupby(
                ['tradingsymbol', 'product', 'variety', 'exchange']
            ).agg(
                average_price=('average_price', 'mean'),
                quantity_total=('quantity', 'sum')
            ).reset_index()
            
            pivot_qty = grouped.pivot_table(
                index=['tradingsymbol', 'product', 'variety', 'exchange'],
                columns='transaction_type',
                values='quantity',
                aggfunc='sum',
                fill_value=0
            ).reset_index()
            
            pivot_df = pivot_qty.merge(
                agg_df, 
                on=['tradingsymbol', 'product', 'variety', 'exchange']
            )
            
            for col in ['BUY', 'SELL']:
                if col not in pivot_df.columns:
                    pivot_df[col] = 0
            
            pivot_df['OPEN_QTY'] = pivot_df['BUY'] - pivot_df['SELL']
            open_positions = pivot_df[pivot_df['OPEN_QTY'] != 0].copy()
            
            if open_positions.empty:
                return None
            
            open_positions['EXIT_QTY'] = open_positions['OPEN_QTY'].abs()
            open_positions['exit_type'] = open_positions['OPEN_QTY'].apply(
                lambda x: 'SELL' if x > 0 else 'BUY'
            )
            
            return open_positions
            
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error fetching positions: {e}")
            return None
    
    # ========================================
    # STEP 2: Display Positions
    # ========================================
    def display_position_buttons(positions_df):
        """Display positions as selectable buttons"""
        with position_output:
            clear_output()
            display(HTML("<h3 style='color: #2196f3;'>📊 Active Positions - Select to Manage</h3>"))
            
            buttons = []
            for idx, row in positions_df.iterrows():
                symbol = row['tradingsymbol']
                exchange = row.get('exchange', 'NFO')
                open_qty = row['OPEN_QTY']
                avg_price = row['average_price']
                side = 'LONG' if open_qty > 0 else 'SHORT'
                side_color = 'green' if open_qty > 0 else 'red'
                
                label = f"{exchange}:{symbol} | {side} {abs(open_qty)} @ ₹{avg_price:.2f}"
                
                btn = widgets.Button(
                    description=label,
                    layout=widgets.Layout(width='auto', margin='5px'),
                    style={'button_color': None}
                )
                
                btn.position_data = row.to_dict()
                btn.on_click(lambda b: select_position(b.position_data))
                
                buttons.append(btn)
            
            rows = [buttons[i:i+2] for i in range(0, len(buttons), 2)]
            for row in rows:
                display(widgets.HBox(row))
            
            refresh_btn = widgets.Button(
                description="🔄 Refresh Positions", 
                button_style="info",
                layout=widgets.Layout(margin='15px 0 0 0')
            )
            refresh_btn.on_click(lambda b: initialize_ui())
            display(refresh_btn)
    
    # ========================================
    # STEP 3: Position Action Panel
    # ========================================
    def select_position(position_data):
        """Show action options for selected position"""
        nonlocal selected_position
        selected_position = position_data
        
        symbol = position_data['tradingsymbol']
        exchange = position_data.get('exchange', 'NFO')
        open_qty = position_data['OPEN_QTY']
        avg_price = position_data['average_price']
        side = 'LONG' if open_qty > 0 else 'SHORT'
        side_color = '#2e7d32' if open_qty > 0 else '#c62828'
        
        with action_panel_output:
            clear_output()
            
            # Compact position header with yellow background
            display(HTML(f"""
                <div style='background: #fff59d; padding: 6px 12px; border-radius: 4px; 
                            margin: 8px 0 5px 0; color: black; font-size: 13px; font-weight: bold;'>
                    {exchange}:{symbol} | 
                    <span style='color: {side_color};'>
                        {side} {abs(open_qty)} @ ₹{avg_price:.2f}
                    </span>
                </div>
            """))
            
            # Compact action buttons
            trail_btn = widgets.Button(
                description="🎯 Trail SL",
                button_style="success",
                layout=widgets.Layout(width='140px', height='35px', margin='5px')
            )
            
            exit_btn = widgets.Button(
                description="⚡ Exit Now",
                button_style="danger",
                layout=widgets.Layout(width='140px', height='35px', margin='5px')
            )
            
            trail_btn.on_click(lambda b: show_trail_config())
            exit_btn.on_click(lambda b: exit_immediately())
            
            display(widgets.HBox(
                [trail_btn, exit_btn],
                layout=widgets.Layout(justify_content='center', margin='10px 0')
            ))
        
        with trail_config_output:
            clear_output()
        
        with trail_status_output:
            clear_output()
        
        with msg_output:
            clear_output()
            print(f"✓ Selected: {exchange}:{symbol} | {side} {abs(open_qty)}")
    
    # ========================================
    # STEP 4: Exit Immediately
    # ========================================
    def exit_immediately():
        """Place market order to exit position immediately"""
        if not selected_position:
            with msg_output:
                clear_output()
                print("⚠️ No position selected!")
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_qty = int(selected_position['EXIT_QTY'])
        exit_type = selected_position['exit_type']
        product = selected_position.get('product', 'MIS')
        variety = selected_position.get('variety', 'regular')
        
        with msg_output:
            clear_output()
            print(f"🚀 Placing MARKET order to exit {exchange}:{symbol}...")
        
        try:
            order_id = kite.place_order(
                variety=variety,
                exchange=exchange,
                tradingsymbol=symbol,
                transaction_type=exit_type,
                quantity=exit_qty,
                product=product,
                order_type="MARKET"
            )
            
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #4caf50; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ✅ EXIT ORDER PLACED SUCCESSFULLY
                        <br>Order ID: {order_id}
                        <br>{exchange}:{symbol} | {exit_type} {exit_qty} @ MARKET
                    </div>
                """))
            
            # Wait and refresh positions
            time.sleep(2)
            initialize_ui()
            
        except Exception as e:
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #f44336; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ❌ ORDER FAILED
                        <br>Error: {str(e)}
                    </div>
                """))
    
    # ========================================
    # STEP 5: Trailing Stop Loss Configuration
    # ========================================
    def show_trail_config():
        """Show trailing stop loss configuration panel"""
        if not selected_position:
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_type = selected_position['exit_type']
        avg_price = selected_position['average_price']
        side = 'LONG' if exit_type == 'SELL' else 'SHORT'
        
        with trail_config_output:
            clear_output()
            
            # Compact config header
            display(HTML(f"""
                <div style='background: #fff3cd; padding: 6px 10px; border-radius: 3px; 
                            border-left: 3px solid #ffc107; margin: 8px 0 5px 0; font-size: 12px;'>
                    <strong>Trail SL Config:</strong> {side} {exchange}:{symbol} @ ₹{avg_price:.2f}
                </div>
            """))
            
            # Configuration inputs - compact
            profit_points_input = widgets.FloatText(
                value=10.0,
                description='Trail Pts:',
                layout=widgets.Layout(width='180px')
            )
            
            check_interval_input = widgets.IntText(
                value=5,
                description='Interval(s):',
                layout=widgets.Layout(width='180px')
            )
            
            manual_mode_checkbox = widgets.Checkbox(
                value=False,
                description='Manual Mode',
                layout=widgets.Layout(width='180px')
            )
            
            display(widgets.HBox([
                profit_points_input,
                check_interval_input,
                manual_mode_checkbox
            ], layout=widgets.Layout(margin='5px 0')))
            
            # Compact Start/Stop buttons
            start_btn = widgets.Button(
                description="▶️ Start",
                button_style="success",
                layout=widgets.Layout(width='120px', margin='5px 3px')
            )
            
            stop_btn = widgets.Button(
                description="⏹️ Stop",
                button_style="danger",
                layout=widgets.Layout(width='120px', margin='5px 3px')
            )
            
            position_key = f"{exchange}:{symbol}"
            
            def start_trailing(b):
                start_trail_stop_loss(
                    profit_points_input.value,
                    check_interval_input.value,
                    manual_mode_checkbox.value
                )
            
            def stop_trailing_sl(b):
                if position_key in stop_trailing:
                    stop_trailing[position_key] = True
                    with msg_output:
                        clear_output()
                        print(f"⏹️ Stopping trailing SL for {position_key}...")
            
            start_btn.on_click(start_trailing)
            stop_btn.on_click(stop_trailing_sl)
            
            display(widgets.HBox([start_btn, stop_btn]))
    
    # ========================================
    # STEP 6: Trailing Stop Loss Logic
    # ========================================
    def start_trail_stop_loss(profit_points, check_interval, manual_mode):
        """Start trailing stop loss for selected position"""
        if not selected_position:
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_qty = int(selected_position['EXIT_QTY'])
        exit_type = selected_position['exit_type']
        avg_price = selected_position['average_price']
        variety = selected_position.get('variety', 'regular')
        product = selected_position.get('product', 'MIS')
        
        position_key = f"{exchange}:{symbol}"
        
        # Check if already trailing
        if position_key in trailing_threads and trailing_threads[position_key].is_alive():
            with msg_output:
                clear_output()
                print(f"⚠️ Trailing SL already active for {position_key}")
            return
        
        # Get initial price
        try:
            instrument = f"{exchange}:{symbol}"
            data = kite.ltp(instrument)
            last_price = data[instrument]["last_price"]
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Failed to get price: {e}")
            return
        
        # Set initial trigger based on position type
        if exit_type == 'SELL':
            # LONG position (BUY entry, SELL exit)
            # Trail below price - use your default mechanism
            trigger_price = last_price - profit_points
        else:
            # SHORT position (SELL entry, BUY exit)
            # Trail above price - INVERSE mechanism
            trigger_price = last_price + profit_points
        
        # First, place the initial SL order
        try:
            order_id = kite.place_order(
                variety=variety,
                exchange=exchange,
                tradingsymbol=symbol,
                transaction_type=exit_type,
                quantity=exit_qty,
                product=product,
                order_type="SL-M",
                trigger_price=trigger_price,
                validity="DAY"
            )
            
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #4caf50; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ✅ TRAILING SL STARTED
                        <br>Order ID: {order_id}
                        <br>Initial Price: ₹{last_price:.2f}
                        <br>Initial Trigger: ₹{trigger_price:.2f}
                        <br>Trailing: {profit_points} points
                    </div>
                """))
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Failed to place SL order: {e}")
            return
        
        # Start trailing thread
        stop_trailing[position_key] = False
        
        def trail_loop():
            nonlocal trigger_price
            current_price = last_price
            
            with trail_status_output:
                clear_output()
                display(HTML(f"<h4 style='color: #2196f3;'>📊 Trailing Status for {position_key}</h4>"))
            
            iteration = 0
            
            while not stop_trailing.get(position_key, False):
                iteration += 1
                
                # Get new price
                if manual_mode:
                    # Manual mode - wait for user input
                    with trail_status_output:
                        print(f"\n[{iteration}] Waiting for manual price input...")
                        print(f"Current: ₹{current_price:.2f} | Trigger: ₹{trigger_price:.2f}")
                    
                    # In manual mode, we'd need input widget - simplified for auto mode
                    time.sleep(check_interval)
                    continue
                else:
                    # Auto mode - fetch live price
                    try:
                        data = kite.ltp(instrument)
                        current_price = data[instrument]["last_price"]
                    except Exception as e:
                        with trail_status_output:
                            print(f"❌ Error fetching price: {e}")
                        time.sleep(check_interval)
                        continue
                
                # Check trailing conditions based on position type
                should_trail = False
                new_trigger = trigger_price
                
                if exit_type == 'SELL':
                    # LONG position - trail UP when price moves UP
                    if current_price > trigger_price + profit_points:
                        extra_points = current_price - trigger_price - profit_points
                        new_trigger = trigger_price + extra_points
                        should_trail = True
                    
                    # Check if SL hit
                    if current_price <= trigger_price:
                        with trail_status_output:
                            print(f"\n⚠️ STOP LOSS HIT!")
                            print(f"Price: ₹{current_price:.2f} <= Trigger: ₹{trigger_price:.2f}")
                        break
                    
                else:
                    # SHORT position - trail DOWN when price moves DOWN (INVERSE)
                    if current_price < trigger_price - profit_points:
                        extra_points = trigger_price - current_price - profit_points
                        new_trigger = trigger_price - extra_points
                        should_trail = True
                    
                    # Check if SL hit
                    if current_price >= trigger_price:
                        with trail_status_output:
                            print(f"\n⚠️ STOP LOSS HIT!")
                            print(f"Price: ₹{current_price:.2f} >= Trigger: ₹{trigger_price:.2f}")
                        break
                
                # Trail the stop loss
                if should_trail:
                    with trail_status_output:
                        print(f"\n▲ TRAILING STOP LOSS")
                        print(f"Price: ₹{current_price:.2f}")
                        print(f"Old Trigger: ₹{trigger_price:.2f} → New: ₹{new_trigger:.2f}")
                    
                    try:
                        new_id = kite.modify_order(
                            variety=variety,
                            order_id=order_id,
                            quantity=exit_qty,
                            product=product,
                            order_type="SL-M",
                            trigger_price=new_trigger,
                            validity="DAY"
                        )
                        trigger_price = new_trigger
                        with trail_status_output:
                            print(f"✓ Modified: {order_id} → {new_id}")
                    except Exception as e:
                        with trail_status_output:
                            print(f"✗ Failed to modify: {e}")
                else:
                    distance = abs(current_price - trigger_price)
                    with trail_status_output:
                        print(f"[{iteration}] Price: ₹{current_price:.2f} | Trigger: ₹{trigger_price:.2f} | Dist: {distance:.2f}")
                
                time.sleep(check_interval)
            
            with trail_status_output:
                print(f"\n⏹️ Trailing SL stopped for {position_key}")
        
        # Start thread
        thread = threading.Thread(target=trail_loop, daemon=True)
        trailing_threads[position_key] = thread
        thread.start()
    
    # ========================================
    # Initialize UI
    # ========================================
    def initialize_ui():
        """Initialize the UI"""
        nonlocal selected_position
        selected_position = None
        
        positions_df = get_aggregated_positions()
        
        with position_output:
            clear_output()
            if positions_df is None:
                print("⚠️ No active positions found")
                return
        
        with action_panel_output:
            clear_output()
        
        with trail_config_output:
            clear_output()
        
        with trail_status_output:
            clear_output()
        
        with msg_output:
            clear_output()
        
        display_position_buttons(positions_df)
    
    # Display UI
    # display(HTML("<h2 style='color: #667eea;'>🎯 Position Exit Manager with Trailing Stop Loss</h2>"))
    display(position_output)
    display(action_panel_output)
    display(trail_config_output)
    display(msg_output)
    display(trail_status_output)
    
    initialize_ui()


# Usage:
# position_exit_manager_with_trail_sl(kite)

# position_exit_manager_with_trail_sl_and_ticker_and uses positions instead orders

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
import threading
from kiteconnect import KiteTicker

# ========================================
# GLOBAL TICKER STATE (shared across all function calls)
# ========================================
_GLOBAL_TICKER = None
_GLOBAL_TICKER_CONNECTED = False
_GLOBAL_LIVE_PRICES = {}
_GLOBAL_TRAILING_ACTIVE = {}

def position_exit_manager_with_trail_sl_and_ticker_2222(kite, api_key):
    """
    Interactive position manager with Real-Time Trailing Stop Loss using WebSocket.
    
    Features:
    - Real-time price updates via Kite Ticker (WebSocket)
    - No API rate limits - instant price updates
    - Trail Stop Loss: Automated trailing mechanism (BUY default, inverse for SELL)
    - Exit Immediately: Market order exit
    - Checks for existing pending orders before placing new ones
    - ✅ FIXED: Shared ticker across multiple function calls
    
    Args:
        kite: KiteConnect instance
        api_key: Your Kite API key (required for ticker)
    """
    
    # Use global ticker state
    global _GLOBAL_TICKER, _GLOBAL_TICKER_CONNECTED, _GLOBAL_LIVE_PRICES, _GLOBAL_TRAILING_ACTIVE
    
    # Constants
    EXECUTED_STATUSES = ['COMPLETE', '']
    REQUIRED_COLUMNS = ['status', 'tradingsymbol', 'order_type', 'variety', 'transaction_type', 
                       'product', 'quantity', 'price', 'average_price', 'exchange']
    
    # State (local to this UI instance)
    selected_position = None
    
    # Output widgets
    position_output = widgets.Output()
    action_panel_output = widgets.Output()
    trail_config_output = widgets.Output()
    trail_status_output = widgets.Output()
    msg_output = widgets.Output()
    ticker_status_output = widgets.Output()
    
    # ========================================
    # Helper: Round to Tick Size
    # ========================================
    def round_to_tick_size(price, tick_size=0.05):
        """Round price to nearest tick size"""
        return round(price / tick_size) * tick_size
    
    def get_tick_size(exchange, tradingsymbol):
        """Get tick size for instrument (default to 0.05 for most stocks/F&O)"""
        try:
            instruments = kite.instruments(exchange)
            instrument_df = pd.DataFrame(instruments)
            match = instrument_df[instrument_df['tradingsymbol'] == tradingsymbol]
            
            if not match.empty:
                tick_size = match.iloc[0].get('tick_size', 0.05)
                return tick_size
            return 0.05  # Default tick size
        except:
            return 0.05  # Default tick size
    
    # ========================================
    # STEP 1: Fetch Positions (IMPROVED - Use Positions API)
    # ========================================
    def get_aggregated_positions():
        """Fetch positions from Kite Positions API (more accurate than order history)"""
        try:
            # Try to get positions from positions API first
            positions_data = kite.positions()
            
            # Combine net and day positions
            all_positions = []
            if positions_data.get('net'):
                all_positions.extend(positions_data['net'])
            if positions_data.get('day'):
                # Add day positions that aren't already in net
                net_symbols = {(p['tradingsymbol'], p['product']) for p in positions_data.get('net', [])}
                for day_pos in positions_data['day']:
                    if (day_pos['tradingsymbol'], day_pos['product']) not in net_symbols:
                        all_positions.append(day_pos)
            
            if not all_positions:
                # Fallback to order-based calculation
                return get_positions_from_orders()
            
            positions_df = pd.DataFrame(all_positions)
            
            # Filter only open positions
            open_positions = positions_df[positions_df['quantity'] != 0].copy()
            
            if open_positions.empty:
                # Fallback to order-based calculation
                return get_positions_from_orders()
            
            # Rename columns to match expected format
            open_positions = open_positions.rename(columns={
                'quantity': 'OPEN_QTY',
                'average_price': 'average_price'
            })
            
            # Add required columns
            open_positions['EXIT_QTY'] = open_positions['OPEN_QTY'].abs()
            open_positions['exit_type'] = open_positions['OPEN_QTY'].apply(
                lambda x: 'SELL' if x > 0 else 'BUY'
            )
            open_positions['variety'] = 'regular'  # Standard variety for exits
            
            # Ensure exchange column exists
            if 'exchange' not in open_positions.columns:
                open_positions['exchange'] = 'NSE'  # Default to NSE
            
            return open_positions[['tradingsymbol', 'product', 'variety', 'exchange', 
                                  'average_price', 'OPEN_QTY', 'EXIT_QTY', 'exit_type']]
            
        except Exception as e:
            with msg_output:
                print(f"⚠️ Could not fetch positions API, using order-based method: {e}")
            return get_positions_from_orders()
    
    def get_positions_from_orders():
        """Fallback: Calculate positions from order history (old method)"""
        try:
            orders_df = pd.DataFrame(kite.orders())
            
            if orders_df.empty:
                return None
            
            executed_orders = orders_df[orders_df['status'].isin(EXECUTED_STATUSES)][REQUIRED_COLUMNS].copy()
            
            if executed_orders.empty:
                return None
            
            grouped = executed_orders.groupby(
                ['tradingsymbol', 'order_type', 'transaction_type', 'product', 'variety', 'exchange'],
                as_index=False
            ).agg({
                'price': 'mean',
                'average_price': 'mean',
                'quantity': 'sum'
            })
            
            agg_df = grouped.groupby(
                ['tradingsymbol', 'product', 'variety', 'exchange']
            ).agg(
                average_price=('average_price', 'mean'),
                quantity_total=('quantity', 'sum')
            ).reset_index()
            
            pivot_qty = grouped.pivot_table(
                index=['tradingsymbol', 'product', 'variety', 'exchange'],
                columns='transaction_type',
                values='quantity',
                aggfunc='sum',
                fill_value=0
            ).reset_index()
            
            pivot_df = pivot_qty.merge(
                agg_df, 
                on=['tradingsymbol', 'product', 'variety', 'exchange']
            )
            
            for col in ['BUY', 'SELL']:
                if col not in pivot_df.columns:
                    pivot_df[col] = 0
            
            pivot_df['OPEN_QTY'] = pivot_df['BUY'] - pivot_df['SELL']
            open_positions = pivot_df[pivot_df['OPEN_QTY'] != 0].copy()
            
            if open_positions.empty:
                return None
            
            open_positions['EXIT_QTY'] = open_positions['OPEN_QTY'].abs()
            open_positions['exit_type'] = open_positions['OPEN_QTY'].apply(
                lambda x: 'SELL' if x > 0 else 'BUY'
            )
            
            return open_positions
            
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error fetching positions: {e}")
            return None
    
    # ========================================
    # Check for Existing Orders
    # ========================================
    def get_existing_pending_order(symbol, exchange, exit_type):
        """Check if there's already a pending SL order for this position"""
        try:
            orders = kite.orders()
            orders_df = pd.DataFrame(orders)
            
            if orders_df.empty:
                return None
            
            # Find pending SL orders for this symbol
            pending_sl = orders_df[
                (orders_df['tradingsymbol'] == symbol) &
                (orders_df['exchange'] == exchange) &
                (orders_df['transaction_type'] == exit_type) &
                (orders_df['order_type'].isin(['SL', 'SL-M'])) &
                (orders_df['status'].isin(['TRIGGER PENDING', 'OPEN']))
            ]
            
            if not pending_sl.empty:
                # Return the most recent pending order
                latest_order = pending_sl.iloc[-1]
                return {
                    'order_id': latest_order['order_id'],
                    'trigger_price': latest_order.get('trigger_price', 0),
                    'quantity': latest_order['quantity'],
                    'variety': latest_order['variety'],
                    'product': latest_order['product']
                }
            
            return None
            
        except Exception as e:
            with msg_output:
                print(f"⚠️ Error checking existing orders: {e}")
            return None
    
    def cancel_existing_order(order_id, variety):
        """Cancel an existing order"""
        try:
            kite.cancel_order(variety=variety, order_id=order_id)
            return True
        except Exception as e:
            with msg_output:
                print(f"⚠️ Failed to cancel order {order_id}: {e}")
            return False
    
    # ========================================
    # STEP 2: Display Positions
    # ========================================
    def display_position_buttons(positions_df):
        """Display positions as selectable buttons"""
        with position_output:
            clear_output()
            display(HTML("<h3 style='color: #2196f3;'>📊 Active Positions - Select to Manage</h3>"))
            
            buttons = []
            for idx, row in positions_df.iterrows():
                symbol = row['tradingsymbol']
                exchange = row.get('exchange', 'NFO')
                open_qty = row['OPEN_QTY']
                avg_price = row['average_price']
                product = row.get('product', 'MIS')  # Get current product type
                side = 'LONG' if open_qty > 0 else 'SHORT'
                side_color = 'green' if open_qty > 0 else 'red'
                
                # Check if this position has active trailing
                position_key = f"{exchange}:{symbol}"
                trailing_indicator = " 🎯" if position_key in _GLOBAL_TRAILING_ACTIVE else ""
                
                label = f"{exchange}:{symbol} | {side} {abs(open_qty)} @ ₹{avg_price:.2f} [{product}]{trailing_indicator}"
                
                btn = widgets.Button(
                    description=label,
                    layout=widgets.Layout(width='auto', margin='5px'),
                    style={'button_color': None}
                )
                
                btn.position_data = row.to_dict()
                btn.on_click(lambda b: select_position(b.position_data))
                
                buttons.append(btn)
            
            rows = [buttons[i:i+2] for i in range(0, len(buttons), 2)]
            for row in rows:
                display(widgets.HBox(row))
            
            refresh_btn = widgets.Button(
                description="🔄 Refresh Positions", 
                button_style="info",
                layout=widgets.Layout(margin='15px 0 0 0')
            )
            refresh_btn.on_click(lambda b: initialize_ui())
            display(refresh_btn)
    
    # ========================================
    # STEP 3: Kite Ticker Setup (GLOBAL/SHARED)
    # ========================================
    def setup_ticker():
        """Initialize Kite Ticker WebSocket - SHARED globally across all function calls"""
        global _GLOBAL_TICKER, _GLOBAL_TICKER_CONNECTED
        
        # If ticker already exists and connected, reuse it
        if _GLOBAL_TICKER is not None and _GLOBAL_TICKER_CONNECTED:
            with ticker_status_output:
                clear_output()
                display(HTML("""
                    <div style='background: #4caf50; color: white; padding: 8px; 
                                border-radius: 4px; font-size: 12px; font-weight: bold;'>
                        🟢 WebSocket Connected (Reusing existing connection)
                    </div>
                """))
            return _GLOBAL_TICKER
        
        # If ticker exists but not connected, clean it up first
        if _GLOBAL_TICKER is not None:
            try:
                _GLOBAL_TICKER.close()
            except:
                pass
            _GLOBAL_TICKER = None
            _GLOBAL_TICKER_CONNECTED = False
        
        # Create new ticker instance
        _GLOBAL_TICKER = KiteTicker(api_key, kite.access_token)
        
        def on_ticks(ws, ticks):
            for tick in ticks:
                instrument_token = tick['instrument_token']
                last_price = tick['last_price']
                _GLOBAL_LIVE_PRICES[instrument_token] = last_price
                
                # Check all active trailing positions
                for pos_key, details in list(_GLOBAL_TRAILING_ACTIVE.items()):
                    if details['instrument_token'] == instrument_token:
                        check_and_trail(pos_key, last_price, details)
        
        def on_connect(ws, response):
            global _GLOBAL_TICKER_CONNECTED
            _GLOBAL_TICKER_CONNECTED = True
            with ticker_status_output:
                clear_output()
                display(HTML("""
                    <div style='background: #4caf50; color: white; padding: 8px; 
                                border-radius: 4px; font-size: 12px; font-weight: bold;'>
                        🟢 WebSocket Connected - Real-time prices active
                    </div>
                """))
        
        def on_close(ws, code, reason):
            global _GLOBAL_TICKER_CONNECTED
            _GLOBAL_TICKER_CONNECTED = False
            with ticker_status_output:
                clear_output()
                display(HTML("""
                    <div style='background: #f44336; color: white; padding: 8px; 
                                border-radius: 4px; font-size: 12px; font-weight: bold;'>
                        🔴 WebSocket Disconnected
                    </div>
                """))
        
        def on_error(ws, code, reason):
            with ticker_status_output:
                print(f"⚠️ Ticker Error [{code}]: {reason}")
        
        _GLOBAL_TICKER.on_ticks = on_ticks
        _GLOBAL_TICKER.on_connect = on_connect
        _GLOBAL_TICKER.on_close = on_close
        _GLOBAL_TICKER.on_error = on_error
        
        # Start ticker in threaded mode
        _GLOBAL_TICKER.connect(threaded=True)
        
        # Wait for connection (max 3 seconds)
        for _ in range(30):
            if _GLOBAL_TICKER_CONNECTED:
                break
            time.sleep(0.1)
        
        return _GLOBAL_TICKER
    
    def subscribe_instrument(exchange, tradingsymbol):
        """Subscribe to instrument for real-time prices"""
        try:
            # Get instrument token
            instruments = kite.instruments(exchange)
            instrument_df = pd.DataFrame(instruments)
            match = instrument_df[instrument_df['tradingsymbol'] == tradingsymbol]
            
            if match.empty:
                with msg_output:
                    print(f"⚠️ Instrument not found: {exchange}:{tradingsymbol}")
                return None
            
            instrument_token = int(match.iloc[0]['instrument_token'])
            
            # Subscribe to ticker
            if _GLOBAL_TICKER and _GLOBAL_TICKER_CONNECTED:
                _GLOBAL_TICKER.subscribe([instrument_token])
                _GLOBAL_TICKER.set_mode(_GLOBAL_TICKER.MODE_LTP, [instrument_token])
                
                with ticker_status_output:
                    print(f"✓ Subscribed: {exchange}:{tradingsymbol} (Token: {instrument_token})")
                
                return instrument_token
            else:
                with msg_output:
                    print("⚠️ Ticker not connected")
                return None
                
        except Exception as e:
            with msg_output:
                print(f"❌ Error subscribing: {e}")
            return None
    
    # ========================================
    # STEP 4: Position Action Panel
    # ========================================
    def select_position(position_data):
        """Show action options for selected position"""
        nonlocal selected_position
        selected_position = position_data
        
        symbol = position_data['tradingsymbol']
        exchange = position_data.get('exchange', 'NFO')
        open_qty = position_data['OPEN_QTY']
        avg_price = position_data['average_price']
        product = position_data.get('product', 'MIS')  # Get current product type
        side = 'LONG' if open_qty > 0 else 'SHORT'
        side_color = '#2e7d32' if open_qty > 0 else '#c62828'
        
        with action_panel_output:
            clear_output()
            
            # Compact position header with yellow background - show product type
            display(HTML(f"""
                <div style='background: #fff59d; padding: 6px 12px; border-radius: 4px; 
                            margin: 8px 0 5px 0; color: black; font-size: 13px; font-weight: bold;'>
                    {exchange}:{symbol} | 
                    <span style='color: {side_color};'>
                        {side} {abs(open_qty)} @ ₹{avg_price:.2f}
                    </span>
                    <span style='background: #2196f3; color: white; padding: 2px 6px; 
                                 border-radius: 3px; margin-left: 8px; font-size: 11px;'>
                        {product}
                    </span>
                </div>
            """))
            
            # Compact action buttons
            trail_btn = widgets.Button(
                description="🎯 Trail SL",
                button_style="success",
                layout=widgets.Layout(width='140px', height='35px', margin='5px')
            )
            
            exit_btn = widgets.Button(
                description="⚡ Exit Now",
                button_style="danger",
                layout=widgets.Layout(width='140px', height='35px', margin='5px')
            )
            
            trail_btn.on_click(lambda b: show_trail_config())
            exit_btn.on_click(lambda b: exit_immediately())
            
            display(widgets.HBox(
                [trail_btn, exit_btn],
                layout=widgets.Layout(justify_content='center', margin='10px 0')
            ))
        
        with trail_config_output:
            clear_output()
        
        with trail_status_output:
            clear_output()
        
        with msg_output:
            clear_output()
            print(f"✓ Selected: {exchange}:{symbol} | {side} {abs(open_qty)}")
    
    # ========================================
    # STEP 5: Exit Immediately
    # ========================================
    def exit_immediately():
        """Place market order to exit position immediately"""
        if not selected_position:
            with msg_output:
                clear_output()
                print("⚠️ No position selected!")
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_qty = int(selected_position['EXIT_QTY'])
        exit_type = selected_position['exit_type']
        product = selected_position.get('product', 'MIS')
        variety = selected_position.get('variety', 'regular')
        position_key = f"{exchange}:{symbol}"
        
        # Cancel any existing trailing orders
        if position_key in _GLOBAL_TRAILING_ACTIVE:
            del _GLOBAL_TRAILING_ACTIVE[position_key]
        
        # Cancel any pending SL orders
        existing_order = get_existing_pending_order(symbol, exchange, exit_type)
        if existing_order:
            cancel_existing_order(existing_order['order_id'], existing_order['variety'])
        
        with msg_output:
            clear_output()
            print(f"🚀 Placing MARKET order to exit {exchange}:{symbol}...")
        
        try:
            order_id = kite.place_order(
                variety=variety,
                exchange=exchange,
                tradingsymbol=symbol,
                transaction_type=exit_type,
                quantity=exit_qty,
                product=product,
                order_type="MARKET"
            )
            
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #4caf50; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ✅ EXIT ORDER PLACED SUCCESSFULLY
                        <br>Order ID: {order_id}
                        <br>{exchange}:{symbol} | {exit_type} {exit_qty} @ MARKET
                    </div>
                """))
            
            # Wait and refresh positions
            time.sleep(2)
            initialize_ui()
            
        except Exception as e:
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #f44336; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ❌ ORDER FAILED
                        <br>Error: {str(e)}
                    </div>
                """))
    
    # ========================================
    # STEP 6: Trailing Stop Loss Configuration
    # ========================================
    def show_trail_config():
        """Show trailing stop loss configuration panel"""
        if not selected_position:
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_type = selected_position['exit_type']
        avg_price = selected_position['average_price']
        side = 'LONG' if exit_type == 'SELL' else 'SHORT'
        position_key = f"{exchange}:{symbol}"
        
        # Check if there's an existing pending order
        existing_order = get_existing_pending_order(symbol, exchange, exit_type)
        
        # Check if trailing is already active for this position
        is_trailing_active = position_key in _GLOBAL_TRAILING_ACTIVE
        
        with trail_config_output:
            clear_output()
            
            # Show status if trailing is active
            if is_trailing_active:
                trail_info = _GLOBAL_TRAILING_ACTIVE[position_key]
                display(HTML(f"""
                    <div style='background: #e8f5e9; padding: 10px; border-radius: 4px; 
                                border-left: 4px solid #4caf50; margin: 8px 0; font-size: 12px;'>
                        ✅ <strong>Trailing Active!</strong>
                        <br>Order ID: {trail_info['order_id']}
                        <br>Current Trigger: ₹{trail_info['trigger_price']:.2f}
                        <br>Trail Points: {trail_info['profit_points']}
                    </div>
                """))
            
            # Show warning if existing order found (and not tracking it)
            if existing_order and not is_trailing_active:
                display(HTML(f"""
                    <div style='background: #fff3cd; padding: 10px; border-radius: 4px; 
                                border-left: 4px solid #ff9800; margin: 8px 0; font-size: 12px;'>
                        ⚠️ <strong>Existing SL Order Found!</strong>
                        <br>Order ID: {existing_order['order_id']}
                        <br>Trigger: ₹{existing_order['trigger_price']:.2f}
                        <br>The existing order will be cancelled and replaced with a new trailing SL.
                    </div>
                """))
            
            # Compact config header
            display(HTML(f"""
                <div style='background: #e3f2fd; padding: 6px 10px; border-radius: 3px; 
                            border-left: 3px solid #2196f3; margin: 8px 0 5px 0; font-size: 12px;'>
                    <strong>Trail SL Config (from Avg Price):</strong> {side} {exchange}:{symbol} @ ₹{avg_price:.2f}
                </div>
            """))
            
            # Configuration input - trail points from average price
            profit_points_input = widgets.FloatText(
                value=10.0,
                description='Trail Points:',
                layout=widgets.Layout(width='200px')
            )
            
            display(profit_points_input)
            
            # Compact Start/Stop buttons
            start_btn = widgets.Button(
                description="▶️ Start Real-Time Trail",
                button_style="success",
                layout=widgets.Layout(width='180px', margin='5px 3px')
            )
            
            stop_btn = widgets.Button(
                description="⏹️ Stop & Cancel",
                button_style="danger",
                layout=widgets.Layout(width='130px', margin='5px 3px')
            )
            
            def start_trailing(b):
                start_trail_stop_loss(profit_points_input.value, existing_order)
            
            def stop_trailing_sl(b):
                if position_key in _GLOBAL_TRAILING_ACTIVE:
                    order_id = _GLOBAL_TRAILING_ACTIVE[position_key]['order_id']
                    variety = _GLOBAL_TRAILING_ACTIVE[position_key]['variety']
                    
                    # Cancel the order
                    if cancel_existing_order(order_id, variety):
                        del _GLOBAL_TRAILING_ACTIVE[position_key]
                        with msg_output:
                            clear_output()
                            print(f"⏹️ Stopped trailing SL and cancelled order {order_id}")
                        with trail_status_output:
                            clear_output()
                        # Refresh position list to remove trailing indicator
                        positions_df = get_aggregated_positions()
                        if positions_df is not None:
                            display_position_buttons(positions_df)
                        # Re-show the config panel
                        show_trail_config()
                    else:
                        with msg_output:
                            clear_output()
                            print(f"⚠️ Failed to cancel order. Removed from tracking.")
                        del _GLOBAL_TRAILING_ACTIVE[position_key]
                else:
                    with msg_output:
                        clear_output()
                        print(f"⚠️ No active trailing for {position_key}")
            
            start_btn.on_click(start_trailing)
            stop_btn.on_click(stop_trailing_sl)
            
            display(widgets.HBox([start_btn, stop_btn]))
    
    # ========================================
    # STEP 7: Real-Time Trailing Stop Loss
    # ========================================
    def start_trail_stop_loss(profit_points, existing_order=None):
        """Start real-time trailing stop loss using WebSocket - FROM AVERAGE PRICE"""
        if not selected_position:
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_qty = int(selected_position['EXIT_QTY'])
        exit_type = selected_position['exit_type']
        avg_price = selected_position['average_price']  # ✅ Use average price
        variety = selected_position.get('variety', 'regular')
        product = selected_position.get('product', 'MIS')
        
        position_key = f"{exchange}:{symbol}"
        
        # Check if already trailing this position
        if position_key in _GLOBAL_TRAILING_ACTIVE:
            with msg_output:
                clear_output()
                print(f"⚠️ Already trailing {position_key}. Stop current trail first.")
            return
        
        # Cancel existing order if present
        if existing_order:
            with msg_output:
                clear_output()
                print(f"🔄 Cancelling existing order {existing_order['order_id']}...")
            cancel_existing_order(existing_order['order_id'], existing_order['variety'])
            time.sleep(0.5)
        
        # Setup ticker if not already
        setup_ticker()
        
        # Subscribe to instrument
        instrument_token = subscribe_instrument(exchange, symbol)
        if not instrument_token:
            return
        
        # Get tick size for this instrument
        tick_size = get_tick_size(exchange, symbol)
        
        # ✅ Calculate initial trigger from AVERAGE PRICE, not LTP
        if exit_type == 'SELL':
            # LONG position - trail below average price
            trigger_price = avg_price - profit_points
        else:
            # SHORT position - trail above average price
            trigger_price = avg_price + profit_points
        
        # ✅ Round to tick size to avoid rejection
        trigger_price = round_to_tick_size(trigger_price, tick_size)
        
        # Place initial SL order
        try:
            order_id = kite.place_order(
                variety=variety,
                exchange=exchange,
                tradingsymbol=symbol,
                transaction_type=exit_type,
                quantity=exit_qty,
                product=product,
                order_type="SL-M",
                trigger_price=trigger_price,
                validity="DAY"
            )
            
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #4caf50; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ✅ REAL-TIME TRAILING SL STARTED
                        <br>Order ID: {order_id}
                        <br>Avg Entry Price: ₹{avg_price:.2f}
                        <br>Initial Trigger: ₹{trigger_price:.2f}
                        <br>Trailing: {profit_points} points from avg price
                        <br>Mode: WebSocket (Instant Updates)
                    </div>
                """))
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Failed to place SL order: {e}")
            return
        
        # Store trailing details in GLOBAL state
        _GLOBAL_TRAILING_ACTIVE[position_key] = {
            'instrument_token': instrument_token,
            'order_id': order_id,
            'trigger_price': trigger_price,
            'profit_points': profit_points,
            'exit_type': exit_type,
            'exit_qty': exit_qty,
            'variety': variety,
            'product': product,
            'symbol': symbol,
            'exchange': exchange,
            'avg_price': avg_price,
            'tick_size': tick_size,  # ✅ Store tick size for trailing
            'last_update': time.time(),
            'update_count': 0,
            'kite': kite,  # Store kite instance for order modifications
            'status_output': trail_status_output  # ✅ Store the output widget reference
        }
        
        with trail_status_output:
            clear_output()
            display(HTML(f"<h4 style='color: #2196f3;'>📊 Real-Time Trailing for {position_key}</h4>"))
            print(f"Entry: ₹{avg_price:.2f} | Initial SL: ₹{trigger_price:.2f}")
            print(f"Waiting for tick updates...")
        
        # Refresh only the position list to show trailing indicator (don't clear current panel)
        positions_df = get_aggregated_positions()
        if positions_df is not None:
            display_position_buttons(positions_df)
    
    def check_and_trail(position_key, current_price, details):
        """Check and trail stop loss based on real-time price"""
        trigger_price = details['trigger_price']
        profit_points = details['profit_points']
        exit_type = details['exit_type']
        order_id = details['order_id']
        avg_price = details.get('avg_price', 0)
        tick_size = details.get('tick_size', 0.05)  # ✅ Get stored tick size
        kite_instance = details.get('kite')
        status_output = details.get('status_output')  # ✅ Get the output widget
        
        if not kite_instance:
            return
        
        # Throttle updates - only update status display every 2 seconds
        current_time = time.time()
        if current_time - details['last_update'] < 2:
            return
        
        details['last_update'] = current_time
        details['update_count'] += 1
        
        should_trail = False
        new_trigger = trigger_price
        
        if exit_type == 'SELL':
            # LONG position - trail UP when price moves UP
            if current_price > trigger_price + profit_points:
                extra_points = current_price - trigger_price - profit_points
                new_trigger = trigger_price + extra_points
                new_trigger = round_to_tick_size(new_trigger, tick_size)  # ✅ Round to tick size
                should_trail = True
            
            # Check if SL hit
            if current_price <= trigger_price:
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"\n⚠️ STOP LOSS HIT!")
                        print(f"Price: ₹{current_price:.2f} <= Trigger: ₹{trigger_price:.2f}")
                if position_key in _GLOBAL_TRAILING_ACTIVE:
                    del _GLOBAL_TRAILING_ACTIVE[position_key]
                return
        else:
            # SHORT position - trail DOWN when price moves DOWN
            if current_price < trigger_price - profit_points:
                extra_points = trigger_price - current_price - profit_points
                new_trigger = trigger_price - extra_points
                new_trigger = round_to_tick_size(new_trigger, tick_size)  # ✅ Round to tick size
                should_trail = True
            
            # Check if SL hit
            if current_price >= trigger_price:
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"\n⚠️ STOP LOSS HIT!")
                        print(f"Price: ₹{current_price:.2f} >= Trigger: ₹{trigger_price:.2f}")
                if position_key in _GLOBAL_TRAILING_ACTIVE:
                    del _GLOBAL_TRAILING_ACTIVE[position_key]
                return
        
        # Calculate P&L
        if exit_type == 'SELL':
            pnl = current_price - avg_price
        else:
            pnl = avg_price - current_price
        pnl_points = pnl
        
        # Trail the stop loss
        if should_trail:
            if status_output:  # ✅ Use stored output widget
                with status_output:
                    print(f"\n▲ TRAILING [{details['update_count']}]")
                    print(f"Price: ₹{current_price:.2f} | P&L: {pnl_points:+.2f} pts")
                    print(f"Old Trigger: ₹{trigger_price:.2f} → New: ₹{new_trigger:.2f}")
            
            try:
                new_id = kite_instance.modify_order(
                    variety=details['variety'],
                    order_id=order_id,
                    quantity=details['exit_qty'],
                    order_type="SL-M",
                    trigger_price=new_trigger,
                    validity="DAY"
                )
                details['trigger_price'] = new_trigger
                details['order_id'] = new_id
                _GLOBAL_TRAILING_ACTIVE[position_key] = details
                
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"✓ Modified: {order_id} → {new_id}")
            except Exception as e:
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"✗ Failed to modify: {e}")
        else:
            distance = abs(current_price - trigger_price)
            if status_output:  # ✅ Use stored output widget
                with status_output:
                    print(f"[{details['update_count']}] LTP: ₹{current_price:.2f} | "
                          f"SL: ₹{trigger_price:.2f} | Dist: {distance:.2f} | "
                          f"P&L: {pnl_points:+.2f}")
    
    # ========================================
    # Initialize UI
    # ========================================
    def initialize_ui():
        """Initialize the UI"""
        nonlocal selected_position
        selected_position = None
        
        positions_df = get_aggregated_positions()
        
        with position_output:
            clear_output()
            if positions_df is None:
                print("⚠️ No active positions found")
                return
        
        with action_panel_output:
            clear_output()
        
        with trail_config_output:
            clear_output()
        
        with trail_status_output:
            clear_output()
        
        with msg_output:
            clear_output()
        
        display_position_buttons(positions_df)
    
    # Display UI
    display(HTML("<h2 style='color: #667eea;'>🎯 Position Exit Manager - Real-Time Trailing (WebSocket)</h2>"))
    display(ticker_status_output)
    display(position_output)
    display(action_panel_output)
    display(trail_config_output)
    display(msg_output)
    display(trail_status_output)
    
    initialize_ui()


# Usage:
# position_exit_manager_with_trail_sl_and_ticker_2222(kite, api_key="your_api_key_here")

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
import threading
from kiteconnect import KiteTicker

# ========================================
# GLOBAL TICKER STATE (shared across all function calls)
# ========================================
_GLOBAL_TICKER = None
_GLOBAL_TICKER_CONNECTED = False
_GLOBAL_LIVE_PRICES = {}
_GLOBAL_TRAILING_ACTIVE = {}

def position_exit_manager_with_trail_sl_and_ticker(kite, api_key):
    """
    Interactive position manager with Real-Time Trailing Stop Loss using WebSocket.
    
    Features:
    - Real-time price updates via Kite Ticker (WebSocket)
    - No API rate limits - instant price updates
    - Trail Stop Loss: Automated trailing mechanism (BUY default, inverse for SELL)
    - Exit Immediately: Market order exit
    - Checks for existing pending orders before placing new ones
    - ✅ FIXED: Shared ticker across multiple function calls
    
    Args:
        kite: KiteConnect instance
        api_key: Your Kite API key (required for ticker)
    """
    
    # Use global ticker state
    global _GLOBAL_TICKER, _GLOBAL_TICKER_CONNECTED, _GLOBAL_LIVE_PRICES, _GLOBAL_TRAILING_ACTIVE
    
    # Constants
    EXECUTED_STATUSES = ['COMPLETE', '']
    REQUIRED_COLUMNS = ['status', 'tradingsymbol', 'order_type', 'variety', 'transaction_type', 
                       'product', 'quantity', 'price', 'average_price', 'exchange']
    
    # State (local to this UI instance)
    selected_position = None
    
    # Output widgets
    position_output = widgets.Output()
    action_panel_output = widgets.Output()
    trail_config_output = widgets.Output()
    trail_status_output = widgets.Output()
    msg_output = widgets.Output()
    ticker_status_output = widgets.Output()
    
    # ========================================
    # STEP 1: Fetch Positions
    # ========================================
    def get_aggregated_positions():
        """Fetch orders and calculate aggregated positions"""
        try:
            orders_df = pd.DataFrame(kite.orders())
            
            if orders_df.empty:
                return None
            
            executed_orders = orders_df[orders_df['status'].isin(EXECUTED_STATUSES)][REQUIRED_COLUMNS].copy()
            
            if executed_orders.empty:
                return None
            
            grouped = executed_orders.groupby(
                ['tradingsymbol', 'order_type', 'transaction_type', 'product', 'variety', 'exchange'],
                as_index=False
            ).agg({
                'price': 'mean',
                'average_price': 'mean',
                'quantity': 'sum'
            })
            
            agg_df = grouped.groupby(
                ['tradingsymbol', 'product', 'variety', 'exchange']
            ).agg(
                average_price=('average_price', 'mean'),
                quantity_total=('quantity', 'sum')
            ).reset_index()
            
            pivot_qty = grouped.pivot_table(
                index=['tradingsymbol', 'product', 'variety', 'exchange'],
                columns='transaction_type',
                values='quantity',
                aggfunc='sum',
                fill_value=0
            ).reset_index()
            
            pivot_df = pivot_qty.merge(
                agg_df, 
                on=['tradingsymbol', 'product', 'variety', 'exchange']
            )
            
            for col in ['BUY', 'SELL']:
                if col not in pivot_df.columns:
                    pivot_df[col] = 0
            
            pivot_df['OPEN_QTY'] = pivot_df['BUY'] - pivot_df['SELL']
            open_positions = pivot_df[pivot_df['OPEN_QTY'] != 0].copy()
            
            if open_positions.empty:
                return None
            
            open_positions['EXIT_QTY'] = open_positions['OPEN_QTY'].abs()
            open_positions['exit_type'] = open_positions['OPEN_QTY'].apply(
                lambda x: 'SELL' if x > 0 else 'BUY'
            )
            
            return open_positions
            
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Error fetching positions: {e}")
            return None
    
    # ========================================
    # Check for Existing Orders
    # ========================================
    def get_existing_pending_order(symbol, exchange, exit_type):
        """Check if there's already a pending SL order for this position"""
        try:
            orders = kite.orders()
            orders_df = pd.DataFrame(orders)
            
            if orders_df.empty:
                return None
            
            # Find pending SL orders for this symbol
            pending_sl = orders_df[
                (orders_df['tradingsymbol'] == symbol) &
                (orders_df['exchange'] == exchange) &
                (orders_df['transaction_type'] == exit_type) &
                (orders_df['order_type'].isin(['SL', 'SL-M'])) &
                (orders_df['status'].isin(['TRIGGER PENDING', 'OPEN']))
            ]
            
            if not pending_sl.empty:
                # Return the most recent pending order
                latest_order = pending_sl.iloc[-1]
                return {
                    'order_id': latest_order['order_id'],
                    'trigger_price': latest_order.get('trigger_price', 0),
                    'quantity': latest_order['quantity'],
                    'variety': latest_order['variety'],
                    'product': latest_order['product']
                }
            
            return None
            
        except Exception as e:
            with msg_output:
                print(f"⚠️ Error checking existing orders: {e}")
            return None
    
    def cancel_existing_order(order_id, variety):
        """Cancel an existing order"""
        try:
            kite.cancel_order(variety=variety, order_id=order_id)
            return True
        except Exception as e:
            with msg_output:
                print(f"⚠️ Failed to cancel order {order_id}: {e}")
            return False
    
    # ========================================
    # STEP 2: Display Positions
    # ========================================
    def display_position_buttons(positions_df):
        """Display positions as selectable buttons"""
        with position_output:
            clear_output()
            display(HTML("<h3 style='color: #2196f3;'>📊 Active Positions - Select to Manage</h3>"))
            
            buttons = []
            for idx, row in positions_df.iterrows():
                symbol = row['tradingsymbol']
                exchange = row.get('exchange', 'NFO')
                open_qty = row['OPEN_QTY']
                avg_price = row['average_price']
                side = 'LONG' if open_qty > 0 else 'SHORT'
                side_color = 'green' if open_qty > 0 else 'red'
                
                # Check if this position has active trailing
                position_key = f"{exchange}:{symbol}"
                trailing_indicator = " 🎯" if position_key in _GLOBAL_TRAILING_ACTIVE else ""
                
                label = f"{exchange}:{symbol} | {side} {abs(open_qty)} @ ₹{avg_price:.2f}{trailing_indicator}"
                
                btn = widgets.Button(
                    description=label,
                    layout=widgets.Layout(width='auto', margin='5px'),
                    style={'button_color': None}
                )
                
                btn.position_data = row.to_dict()
                btn.on_click(lambda b: select_position(b.position_data))
                
                buttons.append(btn)
            
            rows = [buttons[i:i+2] for i in range(0, len(buttons), 2)]
            for row in rows:
                display(widgets.HBox(row))
            
            refresh_btn = widgets.Button(
                description="🔄 Refresh Positions", 
                button_style="info",
                layout=widgets.Layout(margin='15px 0 0 0')
            )
            refresh_btn.on_click(lambda b: initialize_ui())
            display(refresh_btn)
    
    # ========================================
    # STEP 3: Kite Ticker Setup (GLOBAL/SHARED)
    # ========================================
    def setup_ticker():
        """Initialize Kite Ticker WebSocket - SHARED globally across all function calls"""
        global _GLOBAL_TICKER, _GLOBAL_TICKER_CONNECTED
        
        # If ticker already exists and connected, reuse it
        if _GLOBAL_TICKER is not None and _GLOBAL_TICKER_CONNECTED:
            with ticker_status_output:
                clear_output()
                display(HTML("""
                    <div style='background: #4caf50; color: white; padding: 8px; 
                                border-radius: 4px; font-size: 12px; font-weight: bold;'>
                        🟢 WebSocket Connected (Reusing existing connection)
                    </div>
                """))
            return _GLOBAL_TICKER
        
        # If ticker exists but not connected, clean it up first
        if _GLOBAL_TICKER is not None:
            try:
                _GLOBAL_TICKER.close()
            except:
                pass
            _GLOBAL_TICKER = None
            _GLOBAL_TICKER_CONNECTED = False
        
        # Create new ticker instance
        _GLOBAL_TICKER = KiteTicker(api_key, kite.access_token)
        
        def on_ticks(ws, ticks):
            for tick in ticks:
                instrument_token = tick['instrument_token']
                last_price = tick['last_price']
                _GLOBAL_LIVE_PRICES[instrument_token] = last_price
                
                # Check all active trailing positions
                for pos_key, details in list(_GLOBAL_TRAILING_ACTIVE.items()):
                    if details['instrument_token'] == instrument_token:
                        check_and_trail(pos_key, last_price, details)
        
        def on_connect(ws, response):
            global _GLOBAL_TICKER_CONNECTED
            _GLOBAL_TICKER_CONNECTED = True
            with ticker_status_output:
                clear_output()
                display(HTML("""
                    <div style='background: #4caf50; color: white; padding: 8px; 
                                border-radius: 4px; font-size: 12px; font-weight: bold;'>
                        🟢 WebSocket Connected - Real-time prices active
                    </div>
                """))
        
        def on_close(ws, code, reason):
            global _GLOBAL_TICKER_CONNECTED
            _GLOBAL_TICKER_CONNECTED = False
            with ticker_status_output:
                clear_output()
                display(HTML("""
                    <div style='background: #f44336; color: white; padding: 8px; 
                                border-radius: 4px; font-size: 12px; font-weight: bold;'>
                        🔴 WebSocket Disconnected
                    </div>
                """))
        
        def on_error(ws, code, reason):
            with ticker_status_output:
                print(f"⚠️ Ticker Error [{code}]: {reason}")
        
        _GLOBAL_TICKER.on_ticks = on_ticks
        _GLOBAL_TICKER.on_connect = on_connect
        _GLOBAL_TICKER.on_close = on_close
        _GLOBAL_TICKER.on_error = on_error
        
        # Start ticker in threaded mode
        _GLOBAL_TICKER.connect(threaded=True)
        
        # Wait for connection (max 3 seconds)
        for _ in range(30):
            if _GLOBAL_TICKER_CONNECTED:
                break
            time.sleep(0.1)
        
        return _GLOBAL_TICKER
    
    def subscribe_instrument(exchange, tradingsymbol):
        """Subscribe to instrument for real-time prices"""
        try:
            # Get instrument token
            instruments = kite.instruments(exchange)
            instrument_df = pd.DataFrame(instruments)
            match = instrument_df[instrument_df['tradingsymbol'] == tradingsymbol]
            
            if match.empty:
                with msg_output:
                    print(f"⚠️ Instrument not found: {exchange}:{tradingsymbol}")
                return None
            
            instrument_token = int(match.iloc[0]['instrument_token'])
            
            # Subscribe to ticker
            if _GLOBAL_TICKER and _GLOBAL_TICKER_CONNECTED:
                _GLOBAL_TICKER.subscribe([instrument_token])
                _GLOBAL_TICKER.set_mode(_GLOBAL_TICKER.MODE_LTP, [instrument_token])
                
                with ticker_status_output:
                    print(f"✓ Subscribed: {exchange}:{tradingsymbol} (Token: {instrument_token})")
                
                return instrument_token
            else:
                with msg_output:
                    print("⚠️ Ticker not connected")
                return None
                
        except Exception as e:
            with msg_output:
                print(f"❌ Error subscribing: {e}")
            return None
    
    # ========================================
    # STEP 4: Position Action Panel
    # ========================================
    def select_position(position_data):
        """Show action options for selected position"""
        nonlocal selected_position
        selected_position = position_data
        
        symbol = position_data['tradingsymbol']
        exchange = position_data.get('exchange', 'NFO')
        open_qty = position_data['OPEN_QTY']
        avg_price = position_data['average_price']
        side = 'LONG' if open_qty > 0 else 'SHORT'
        side_color = '#2e7d32' if open_qty > 0 else '#c62828'
        
        with action_panel_output:
            clear_output()
            
            # Compact position header with yellow background
            display(HTML(f"""
                <div style='background: #fff59d; padding: 6px 12px; border-radius: 4px; 
                            margin: 8px 0 5px 0; color: black; font-size: 13px; font-weight: bold;'>
                    {exchange}:{symbol} | 
                    <span style='color: {side_color};'>
                        {side} {abs(open_qty)} @ ₹{avg_price:.2f}
                    </span>
                </div>
            """))
            
            # Compact action buttons
            trail_btn = widgets.Button(
                description="🎯 Trail SL",
                button_style="success",
                layout=widgets.Layout(width='140px', height='35px', margin='5px')
            )
            
            exit_btn = widgets.Button(
                description="⚡ Exit Now",
                button_style="danger",
                layout=widgets.Layout(width='140px', height='35px', margin='5px')
            )
            
            trail_btn.on_click(lambda b: show_trail_config())
            exit_btn.on_click(lambda b: exit_immediately())
            
            display(widgets.HBox(
                [trail_btn, exit_btn],
                layout=widgets.Layout(justify_content='center', margin='10px 0')
            ))
        
        with trail_config_output:
            clear_output()
        
        with trail_status_output:
            clear_output()
        
        with msg_output:
            clear_output()
            print(f"✓ Selected: {exchange}:{symbol} | {side} {abs(open_qty)}")
    
    # ========================================
    # STEP 5: Exit Immediately
    # ========================================
    def exit_immediately():
        """Place market order to exit position immediately"""
        if not selected_position:
            with msg_output:
                clear_output()
                print("⚠️ No position selected!")
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_qty = int(selected_position['EXIT_QTY'])
        exit_type = selected_position['exit_type']
        product = selected_position.get('product', 'MIS')
        variety = selected_position.get('variety', 'regular')
        position_key = f"{exchange}:{symbol}"
        
        # Cancel any existing trailing orders
        if position_key in _GLOBAL_TRAILING_ACTIVE:
            del _GLOBAL_TRAILING_ACTIVE[position_key]
        
        # Cancel any pending SL orders
        existing_order = get_existing_pending_order(symbol, exchange, exit_type)
        if existing_order:
            cancel_existing_order(existing_order['order_id'], existing_order['variety'])
        
        with msg_output:
            clear_output()
            print(f"🚀 Placing MARKET order to exit {exchange}:{symbol}...")
        
        try:
            order_id = kite.place_order(
                variety=variety,
                exchange=exchange,
                tradingsymbol=symbol,
                transaction_type=exit_type,
                quantity=exit_qty,
                product=product,
                order_type="MARKET"
            )
            
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #4caf50; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ✅ EXIT ORDER PLACED SUCCESSFULLY
                        <br>Order ID: {order_id}
                        <br>{exchange}:{symbol} | {exit_type} {exit_qty} @ MARKET
                    </div>
                """))
            
            # Wait and refresh positions
            time.sleep(2)
            initialize_ui()
            
        except Exception as e:
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #f44336; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ❌ ORDER FAILED
                        <br>Error: {str(e)}
                    </div>
                """))
    
    # ========================================
    # STEP 6: Trailing Stop Loss Configuration
    # ========================================
    def show_trail_config():
        """Show trailing stop loss configuration panel"""
        if not selected_position:
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_type = selected_position['exit_type']
        avg_price = selected_position['average_price']
        side = 'LONG' if exit_type == 'SELL' else 'SHORT'
        position_key = f"{exchange}:{symbol}"
        
        # Check if there's an existing pending order
        existing_order = get_existing_pending_order(symbol, exchange, exit_type)
        
        # Check if trailing is already active for this position
        is_trailing_active = position_key in _GLOBAL_TRAILING_ACTIVE
        
        with trail_config_output:
            clear_output()
            
            # Show status if trailing is active
            if is_trailing_active:
                trail_info = _GLOBAL_TRAILING_ACTIVE[position_key]
                display(HTML(f"""
                    <div style='background: #e8f5e9; padding: 10px; border-radius: 4px; 
                                border-left: 4px solid #4caf50; margin: 8px 0; font-size: 12px;'>
                        ✅ <strong>Trailing Active!</strong>
                        <br>Order ID: {trail_info['order_id']}
                        <br>Current Trigger: ₹{trail_info['trigger_price']:.2f}
                        <br>Trail Points: {trail_info['profit_points']}
                    </div>
                """))
            
            # Show warning if existing order found (and not tracking it)
            if existing_order and not is_trailing_active:
                display(HTML(f"""
                    <div style='background: #fff3cd; padding: 10px; border-radius: 4px; 
                                border-left: 4px solid #ff9800; margin: 8px 0; font-size: 12px;'>
                        ⚠️ <strong>Existing SL Order Found!</strong>
                        <br>Order ID: {existing_order['order_id']}
                        <br>Trigger: ₹{existing_order['trigger_price']:.2f}
                        <br>The existing order will be cancelled and replaced with a new trailing SL.
                    </div>
                """))
            
            # Compact config header
            display(HTML(f"""
                <div style='background: #e3f2fd; padding: 6px 10px; border-radius: 3px; 
                            border-left: 3px solid #2196f3; margin: 8px 0 5px 0; font-size: 12px;'>
                    <strong>Trail SL Config (from Avg Price):</strong> {side} {exchange}:{symbol} @ ₹{avg_price:.2f}
                </div>
            """))
            
            # Configuration input - trail points from average price
            profit_points_input = widgets.FloatText(
                value=10.0,
                description='Trail Points:',
                layout=widgets.Layout(width='200px')
            )
            
            display(profit_points_input)
            
            # Compact Start/Stop buttons
            start_btn = widgets.Button(
                description="▶️ Start Real-Time Trail",
                button_style="success",
                layout=widgets.Layout(width='180px', margin='5px 3px')
            )
            
            stop_btn = widgets.Button(
                description="⏹️ Stop & Cancel",
                button_style="danger",
                layout=widgets.Layout(width='130px', margin='5px 3px')
            )
            
            def start_trailing(b):
                start_trail_stop_loss(profit_points_input.value, existing_order)
            
            def stop_trailing_sl(b):
                if position_key in _GLOBAL_TRAILING_ACTIVE:
                    order_id = _GLOBAL_TRAILING_ACTIVE[position_key]['order_id']
                    variety = _GLOBAL_TRAILING_ACTIVE[position_key]['variety']
                    
                    # Cancel the order
                    if cancel_existing_order(order_id, variety):
                        del _GLOBAL_TRAILING_ACTIVE[position_key]
                        with msg_output:
                            clear_output()
                            print(f"⏹️ Stopped trailing SL and cancelled order {order_id}")
                        with trail_status_output:
                            clear_output()
                        # Refresh position list to remove trailing indicator
                        positions_df = get_aggregated_positions()
                        if positions_df is not None:
                            display_position_buttons(positions_df)
                        # Re-show the config panel
                        show_trail_config()
                    else:
                        with msg_output:
                            clear_output()
                            print(f"⚠️ Failed to cancel order. Removed from tracking.")
                        del _GLOBAL_TRAILING_ACTIVE[position_key]
                else:
                    with msg_output:
                        clear_output()
                        print(f"⚠️ No active trailing for {position_key}")
            
            start_btn.on_click(start_trailing)
            stop_btn.on_click(stop_trailing_sl)
            
            display(widgets.HBox([start_btn, stop_btn]))
    
    # ========================================
    # STEP 7: Real-Time Trailing Stop Loss
    # ========================================
    def start_trail_stop_loss(profit_points, existing_order=None):
        """Start real-time trailing stop loss using WebSocket - FROM AVERAGE PRICE"""
        if not selected_position:
            return
        
        symbol = selected_position['tradingsymbol']
        exchange = selected_position.get('exchange', 'NFO')
        exit_qty = int(selected_position['EXIT_QTY'])
        exit_type = selected_position['exit_type']
        avg_price = selected_position['average_price']  # ✅ Use average price
        variety = selected_position.get('variety', 'regular')
        product = selected_position.get('product', 'MIS')
        
        position_key = f"{exchange}:{symbol}"
        
        # Check if already trailing this position
        if position_key in _GLOBAL_TRAILING_ACTIVE:
            with msg_output:
                clear_output()
                print(f"⚠️ Already trailing {position_key}. Stop current trail first.")
            return
        
        # Cancel existing order if present
        if existing_order:
            with msg_output:
                clear_output()
                print(f"🔄 Cancelling existing order {existing_order['order_id']}...")
            cancel_existing_order(existing_order['order_id'], existing_order['variety'])
            time.sleep(0.5)
        
        # Setup ticker if not already
        setup_ticker()
        
        # Subscribe to instrument
        instrument_token = subscribe_instrument(exchange, symbol)
        if not instrument_token:
            return
        
        # ✅ Calculate initial trigger from AVERAGE PRICE, not LTP
        if exit_type == 'SELL':
            # LONG position - trail below average price
            trigger_price = avg_price - profit_points
        else:
            # SHORT position - trail above average price
            trigger_price = avg_price + profit_points
        
        # Place initial SL order
        try:
            order_id = kite.place_order(
                variety=variety,
                exchange=exchange,
                tradingsymbol=symbol,
                transaction_type=exit_type,
                quantity=exit_qty,
                product=product,
                order_type="SL-M",
                trigger_price=trigger_price,
                validity="DAY"
            )
            
            with msg_output:
                clear_output()
                display(HTML(f"""
                    <div style='background: #4caf50; color: white; padding: 15px; 
                                border-radius: 5px; font-weight: bold;'>
                        ✅ REAL-TIME TRAILING SL STARTED
                        <br>Order ID: {order_id}
                        <br>Avg Entry Price: ₹{avg_price:.2f}
                        <br>Initial Trigger: ₹{trigger_price:.2f}
                        <br>Trailing: {profit_points} points from avg price
                        <br>Mode: WebSocket (Instant Updates)
                    </div>
                """))
        except Exception as e:
            with msg_output:
                clear_output()
                print(f"❌ Failed to place SL order: {e}")
            return
        
        # Store trailing details in GLOBAL state
        _GLOBAL_TRAILING_ACTIVE[position_key] = {
            'instrument_token': instrument_token,
            'order_id': order_id,
            'trigger_price': trigger_price,
            'profit_points': profit_points,
            'exit_type': exit_type,
            'exit_qty': exit_qty,
            'variety': variety,
            'product': product,
            'symbol': symbol,
            'exchange': exchange,
            'avg_price': avg_price,
            'last_update': time.time(),
            'update_count': 0,
            'kite': kite,  # Store kite instance for order modifications
            'status_output': trail_status_output  # ✅ Store the output widget reference
        }
        
        with trail_status_output:
            clear_output()
            display(HTML(f"<h4 style='color: #2196f3;'>📊 Real-Time Trailing for {position_key}</h4>"))
            print(f"Entry: ₹{avg_price:.2f} | Initial SL: ₹{trigger_price:.2f}")
            print(f"Waiting for tick updates...")
        
        # Refresh only the position list to show trailing indicator (don't clear current panel)
        positions_df = get_aggregated_positions()
        if positions_df is not None:
            display_position_buttons(positions_df)
    
    def check_and_trail(position_key, current_price, details):
        """Check and trail stop loss based on real-time price"""
        trigger_price = details['trigger_price']
        profit_points = details['profit_points']
        exit_type = details['exit_type']
        order_id = details['order_id']
        avg_price = details.get('avg_price', 0)
        kite_instance = details.get('kite')
        status_output = details.get('status_output')  # ✅ Get the output widget
        
        if not kite_instance:
            return
        
        # Throttle updates - only update status display every 2 seconds
        current_time = time.time()
        if current_time - details['last_update'] < 2:
            return
        
        details['last_update'] = current_time
        details['update_count'] += 1
        
        should_trail = False
        new_trigger = trigger_price
        
        if exit_type == 'SELL':
            # LONG position - trail UP when price moves UP
            if current_price > trigger_price + profit_points:
                extra_points = current_price - trigger_price - profit_points
                new_trigger = trigger_price + extra_points
                should_trail = True
            
            # Check if SL hit
            if current_price <= trigger_price:
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"\n⚠️ STOP LOSS HIT!")
                        print(f"Price: ₹{current_price:.2f} <= Trigger: ₹{trigger_price:.2f}")
                if position_key in _GLOBAL_TRAILING_ACTIVE:
                    del _GLOBAL_TRAILING_ACTIVE[position_key]
                return
        else:
            # SHORT position - trail DOWN when price moves DOWN
            if current_price < trigger_price - profit_points:
                extra_points = trigger_price - current_price - profit_points
                new_trigger = trigger_price - extra_points
                should_trail = True
            
            # Check if SL hit
            if current_price >= trigger_price:
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"\n⚠️ STOP LOSS HIT!")
                        print(f"Price: ₹{current_price:.2f} >= Trigger: ₹{trigger_price:.2f}")
                if position_key in _GLOBAL_TRAILING_ACTIVE:
                    del _GLOBAL_TRAILING_ACTIVE[position_key]
                return
        
        # Calculate P&L
        if exit_type == 'SELL':
            pnl = current_price - avg_price
        else:
            pnl = avg_price - current_price
        pnl_points = pnl
        
        # Trail the stop loss
        if should_trail:
            if status_output:  # ✅ Use stored output widget
                with status_output:
                    print(f"\n▲ TRAILING [{details['update_count']}]")
                    print(f"Price: ₹{current_price:.2f} | P&L: {pnl_points:+.2f} pts")
                    print(f"Old Trigger: ₹{trigger_price:.2f} → New: ₹{new_trigger:.2f}")
            
            try:
                new_id = kite_instance.modify_order(
                    variety=details['variety'],
                    order_id=order_id,
                    quantity=details['exit_qty'],
                    order_type="SL-M",
                    trigger_price=new_trigger,
                    validity="DAY"
                )
                details['trigger_price'] = new_trigger
                details['order_id'] = new_id
                _GLOBAL_TRAILING_ACTIVE[position_key] = details
                
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"✓ Modified: {order_id} → {new_id}")
            except Exception as e:
                if status_output:  # ✅ Use stored output widget
                    with status_output:
                        print(f"✗ Failed to modify: {e}")
        else:
            distance = abs(current_price - trigger_price)
            if status_output:  # ✅ Use stored output widget
                with status_output:
                    print(f"[{details['update_count']}] LTP: ₹{current_price:.2f} | "
                          f"SL: ₹{trigger_price:.2f} | Dist: {distance:.2f} | "
                          f"P&L: {pnl_points:+.2f}")
    
    # ========================================
    # Initialize UI
    # ========================================
    def initialize_ui():
        """Initialize the UI"""
        nonlocal selected_position
        selected_position = None
        
        positions_df = get_aggregated_positions()
        
        with position_output:
            clear_output()
            if positions_df is None:
                print("⚠️ No active positions found")
                return
        
        with action_panel_output:
            clear_output()
        
        with trail_config_output:
            clear_output()
        
        with trail_status_output:
            clear_output()
        
        with msg_output:
            clear_output()
        
        display_position_buttons(positions_df)
    
    # Display UI
    display(HTML("<h2 style='color: #667eea;'>🎯 Position Exit Manager - Real-Time Trailing (WebSocket)</h2>"))
    display(ticker_status_output)
    display(position_output)
    display(action_panel_output)
    display(trail_config_output)
    display(msg_output)
    display(trail_status_output)
    
    initialize_ui()


# Usage:
# position_exit_manager_with_trail_sl_and_ticker_2222(kite, api_key="your_api_key_here")